# Set up the Environment and read Functions

In [ ]:
import importlib.util
import subprocess
import multiprocessing

def install_packages(packages):
    """
    Check if the required packages are installed, and install them if not.

    Args:
    packages (list): List of package names to install.
    """
    for package in packages:
        spec = importlib.util.find_spec(package)
        if spec is None:
            print(f"{package} is not installed. Installing...")
            subprocess.check_call(["pip", "install", package])
        else:
            print(f"{package} is already installed.")

# List of packages to check/install
required_packages = ["geopandas", "pgeocode", "geopy", "flexitext",'umap','umap-learn','datashader','bokeh','holoviews','scikit-image','colorcet','dask[complete]']

# Call the function to install packages if needed
install_packages(required_packages)
import pandas as pd
import os
import subprocess
from scipy import stats
import statsmodels.stats.multicomp as mc
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import pgeocode
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import hail as hl
import bokeh
from hail.plot import show, output_notebook
from scipy.stats import ttest_ind, probplot, shapiro
from hail.plot import show, output_notebook
from sklearn.decomposition import PCA
import ast
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from tabulate import tabulate
import umap.plot
from sklearn.preprocessing import PowerTransformer
from sklearn.cluster import KMeans
from sklearn.preprocessing import PowerTransformer, OneHotEncoder
!pip install --upgrade datashader
!pip install "dask[complete]"

multiprocessing.cpu_count()
def save(folder_name,filename,file):
    # Replace df with THE NAME OF YOUR DATAFRAME
    my_dataframe = file
    destination_filename = '{}/{}'.format(folder_name,filename)
    # save dataframe in a csv file in the same workspace as the notebook
    my_dataframe.to_csv(destination_filename, index=False)
    # get the bucket name
    my_bucket = os.getenv('WORKSPACE_BUCKET')
    # copy csv file to the bucket
    args = ["gsutil", "cp", f"./{destination_filename}", f"{my_bucket}/data/"]
    output = subprocess.run(args, capture_output=True)
    # print output from gsutil
    output.stderr
def read_tsv(file_name,path='/'):
    # Replace 'test.csv' with THE NAME of the file you're going to download from the bucket (don't delete the quotation marks)
    if path == '/':
        name_of_file_in_bucket = '{}'.format(file_name)
    else:
        name_of_file_in_bucket = '{}/{}'.format(path,file_name)
    # get the bucket name
    my_bucket = os.getenv('WORKSPACE_BUCKET')
    # copy csv file from the bucket to the current working space
    os.system(f"gsutil cp '{my_bucket}/data/{name_of_file_in_bucket}' .")
    print(f'[INFO] {name_of_file_in_bucket} is successfully downloaded into your working space')
    # save dataframe in a csv file in the same workspace as the notebook
    file = pd.read_csv(name_of_file_in_bucket,sep='\t')
    return file
def read_csv(file_name,path='/'):
    # Replace 'test.csv' with THE NAME of the file you're going to download from the bucket (don't delete the quotation marks)
    if path == '/':
        name_of_file_in_bucket = '{}'.format(file_name)
    else:
        name_of_file_in_bucket = '{}/{}'.format(path,file_name)
    # get the bucket name
    my_bucket = os.getenv('WORKSPACE_BUCKET')
    # copy csv file from the bucket to the current working space
    os.system(f"gsutil cp '{my_bucket}/data/{name_of_file_in_bucket}' .")
    print(f'[INFO] {name_of_file_in_bucket} is successfully downloaded into your working space')
    # save dataframe in a csv file in the same workspace as the notebook
    file = pd.read_csv(name_of_file_in_bucket,sep=',')
    return file
def initialize_hail_environment():
    """
    Initialize the Hail environment and return the bucket variable and genomic location.

    Returns:
    str: Bucket variable for storing Hail data.
    str: Genomic location.
    """
    # Initialize Hail
    hl.init(default_reference="GRCh38")
    
    # Retrieve bucket variable and genomic location
    bucket = os.getenv('WORKSPACE_BUCKET')
    genomic_location = os.getenv("CDR_STORAGE_PATH")
    
    return bucket, genomic_location
def read_person_information():
    """
    Reads person information from a given file and filters it to keep only data with ICD10 and ICD9 codes.

    
    Returns:
    pd.DataFrame: DataFrame containing filtered person information.
    str: Message indicating the action taken.
    """
    # This query represents dataset "All participants" for domain "condition" and was generated for All of Us Controlled Tier Dataset v7
    dataset_25030714_condition_sql = """
        SELECT
            c_occurrence.person_id,
            c_occurrence.condition_concept_id,
            c_standard_concept.concept_name as standard_concept_name,
            c_standard_concept.concept_code as standard_concept_code,
            c_standard_concept.vocabulary_id as standard_vocabulary,
            c_occurrence.condition_start_datetime,
            c_occurrence.condition_end_datetime,
            c_occurrence.condition_type_concept_id,
            c_type.concept_name as condition_type_concept_name,
            c_occurrence.stop_reason,
            c_occurrence.visit_occurrence_id,
            visit.concept_name as visit_occurrence_concept_name,
            c_occurrence.condition_source_value,
            c_occurrence.condition_source_concept_id,
            c_source_concept.concept_name as source_concept_name,
            c_source_concept.concept_code as source_concept_code,
            c_source_concept.vocabulary_id as source_vocabulary,
            c_occurrence.condition_status_source_value,
            c_occurrence.condition_status_concept_id,
            c_status.concept_name as condition_status_concept_name 
        FROM
            ( SELECT
                * 
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.condition_occurrence` c_occurrence 
            WHERE
                (
                    condition_concept_id IN  (
                        SELECT
                            DISTINCT c.concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                        JOIN
                            (
                                select
                                    cast(cr.id as string) as id 
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                WHERE
                                    concept_id IN (
                                        133468, 134057, 134736, 135930, 138239, 138525, 140190, 141960, 192438, 193460, 194133, 200174, 200219, 200452, 201618, 201891, 253549, 254068, 254761, 255919, 31610, 316866, 31821, 318800, 320128, 320136, 321588, 37018424, 37018713, 37203927, 37206233, 372887, 37311677, 37311678, 373499, 375252, 376106, 376208, 376337, 4000609, 4000610, 4002898, 4002905, 4006969, 4009890, 4011630, 4020346, 4021667, 4021780, 4021915, 4022201, 4022922, 4022923, 4022924, 4023995, 4024000, 4024013, 4024561, 4024566, 4024567, 4025215, 4027384, 4027553, 4028071, 4028076, 4028387, 4031814, 4038502, 4038677, 4038678, 4041283, 4041284, 4041285, 4041436, 4042056, 4042140, 4042141, 4042142, 4042503, 4042505, 4042835, 4042836, 4042837, 4043346, 4043371, 4043671, 4047779, 40480457, 40481517, 40481841, 40484102, 40484533, 40484935, 4051221, 4054501, 4071689, 4080992, 4081176, 4083787, 4086181, 4090426, 4090614, 4090615, 4091213, 4091363, 4091532, 4093227, 4093228, 4093991, 4094294, 4095793, 4096864, 4100932, 4101212, 4101343, 4101796, 4102111, 4103183, 4103320, 4103352, 4104157, 4113563, 4113999, 4115259, 4115386, 4115390, 4116811, 4117779, 4117930, 4132555, 4132926, 4134294, 4134440, 4150129, 4160062, 4162282, 4168335, 4170143, 4170226, 4170962, 4171379, 4175154, 4178545, 4178680, 4178818, 4179141, 4179167, 4179871, 4179873, 4180154, 4180169, 4180170, 4180628, 4181063, 4181187, 4182161, 4182165, 4182633, 4184252, 4185503, 4185640, 4190185, 4197094, 4198525, 4199402, 4200532, 4201745, 4206591, 4208786, 4212577, 4213101, 4221108, 4227253, 4244662, 4247371, 42538830, 4260918, 4266186, 4267789, 4269314, 4272867, 4274025, 4276569, 4276572, 4277352, 4293175, 4297887, 4301699, 4302537, 4304916, 4305027, 4308811, 4309188, 4316083, 4317258, 4318379, 432250, 432586, 432795, 432867, 4329041, 433128, 433736, 4338120, 4339410, 4339468, 4344497, 43530815, 43530877, 43531053, 43531054, 43531056, 43531057, 43531058, 43531059, 43531060, 435506, 435524, 436096, 436670, 437515, 438112, 440029, 440142, 440371, 440383, 440921, 441542, 442077, 443343, 443723, 443783, 443784, 443883, 444089, 444100, 444108, 444112, 444363, 44782620, 44783587, 44784102, 73553, 75865, 75909, 77074, 77670, 77960, 79106, 80180
                                    ) 
                                    AND full_text LIKE '%_rank1]%'
                            ) a 
                                ON (
                                    c.path LIKE CONCAT('%.',
                                a.id,
                                '.%') 
                                OR c.path LIKE CONCAT('%.',
                                a.id) 
                                OR c.path LIKE CONCAT(a.id,
                                '.%') 
                                OR c.path = a.id) 
                            WHERE
                                is_standard = 1 
                                AND is_selectable = 1
                            ) 
                            OR  condition_source_concept_id IN  (
                                SELECT
                                    DISTINCT c.concept_id 
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                JOIN
                                    (
                                        select
                                            cast(cr.id as string) as id 
                                        FROM
                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                        WHERE
                                            concept_id IN (
                                                1326492, 1326493, 1326788, 1326896, 1326897, 1326898, 1326899, 1553844, 1567858, 1567893, 1567956, 1567958, 1567959, 1567960, 1567964, 1567965, 1567966, 1567969, 1567971, 1568017, 1568022, 1568023, 1568062, 1568177, 1568178, 1568179, 1568180, 1568217, 1568232, 1568233, 1568330, 1568337, 1568339, 1568344, 1568362, 1568363, 1568364, 1568366, 1568420, 1568421, 1568422, 1569133, 1569134, 1569135, 1569145, 1569447, 1569472, 1569473, 1569488, 1569489, 1569490, 1569491, 1569558, 1569639, 1569650, 1569657, 1569658, 1569659, 1569660, 1569661, 1569662, 1569663, 1569664, 1569665, 1569811, 1570333, 1570557, 1570563, 1570565, 1570566, 1570567, 1570568, 1570569, 1570570, 1570663, 1570671, 1570701, 1570703, 1570704, 1570705, 1570790, 1570791, 1570824, 1570832, 1571057, 1571059, 1571060, 1571061, 1571062, 1571063, 1571064, 1571177, 1571515, 1571516, 1571517, 1572190, 1572191, 1572193, 1572198, 1572201, 1572203, 1572209, 1572226, 1572229, 1572255, 1572256, 1572261, 1572273, 1572274, 1572281, 1576304, 1595534, 1595535, 1595617, 1595618, 1595619, 1595620, 1595697, 1595698, 1595699, 1595700, 1595701, 1595798, 35206751, 35206752, 35206753, 35206754, 35206755, 35206757, 35206852, 35206853, 35206854, 35206855, 35206856, 35206857, 35206858, 35206859, 35206881, 35206882, 35206991, 35206992, 35207021, 35207022, 35207023, 35207024, 35207060, 35207061, 35207062, 35207063, 35207064, 35207065, 35207066, 35207067, 35207092, 35207093, 35207094, 35207095, 35207096, 35207097, 35207098, 35207099, 35207100, 35207150, 35207151, 35207152, 35207153, 35207154, 35207155, 35207169, 35207170, 35207171, 35207172, 35207173, 35207175, 35207176, 35207177, 35207381, 35207409, 35207410, 35207543, 35207566, 35207567, 35207568, 35207668, 35207702, 35207703, 35207704, 35207705, 35207706, 35207922, 35207923, 35207924, 35207928, 35207929, 35207971, 35207972, 35207973, 35208189, 35208190, 35208288, 35208289, 35208290, 35208291, 35208292, 35208293, 35208772, 35208773, 35208774, 35208775, 35208776, 35208810, 35208871, 35208968, 35208969, 35208970, 35208971, 35208974, 35208975, 35209005, 35209006, 35209007, 35209008, 35209009, 35209010, 35209011, 35209013, 35209098, 35209099, 35209324, 35209325, 35209326, 35209327, 35211261, 35211262, 35211263, 35211264, 35211275, 35211276, 35211277, 35211278, 35211279, 35211280, 35211281, 35211282, 35211283, 35211284, 35211285, 35211286, 35211287, 35211288, 35211289, 35211290, 35211299, 35211300, 35211301, 35211302, 35211303, 35211304, 35211305, 35211306, 35211307, 35211308, 35211309, 35211336, 35211337, 35211338, 35211350, 35211388, 35211400, 35211401, 35211402, 35211414, 35211415, 35211416, 35211420, 35211442, 35211443, 35211512, 35211513, 35211514, 35211515, 35211516, 35211517, 35211518, 35211519, 35211520, 35211521, 35211522, 35211523, 35211524, 35211525, 35211527, 35225038, 35225264, 35225374, 35225375, 35225376, 35225378, 35225379, 37200198, 37200199, 37200200, 37200201, 37200202, 37200203, 37200204, 37200205, 37200206, 37200207, 37200208, 37200209, 37200210, 37200211, 37200212, 37200213, 37200214, 37200215, 37200216, 37200217, 37200218, 37200219, 37200220, 37200221, 37200222, 37200223, 37200224, 37200225, 37200227, 37200228, 37200229, 37200230, 37200232, 37200233, 37200234, 37200235, 37200237, 37200238, 37200239, 37200240, 37200242, 37200243, 37200244, 37200245, 37200246, 37200247, 37200248, 37200249, 37200251, 37200252, 37200253, 37200254, 37200312, 37200313, 37200319, 37200320, 37200649, 37200650, 37200651, 37200652, 37200701, 37200702, 37200703, 37200744, 37200877, 37200878, 37201113, 37201116, 37201118, 37201119, 37201120, 37201121, 37202304, 37202305, 44819500, 44819501, 44819502, 44819503, 44819504, 44819524, 44819525, 44819544, 44819554, 44819555, 44819556, 44819777, 44819778, 44819819, 44819946, 44819947, 44819948, 44819964, 44819965, 44819966, 44819967, 44819968, 44819970, 44819973, 44819974, 44819975, 44819978, 44819984, 44819985, 44819986, 44819987, 44820052, 44820053, 44820054, 44820055, 44820056, 44820057, 44820059, 44820060, 44820061, 44820062, 44820063, 44820064, 44820065, 44820066, 44820067, 44820068, 44820069, 44820070, 44820489, 44820682, 44820683, 44820684, 44820685, 44820725, 44820730, 44820731, 44820732, 44820733, 44820889, 44820924, 44820925, 44820926, 44820979, 44820980, 44820981, 44821091, 44821092, 44821101, 44821125, 44821142, 44821143, 44821144, 44821145, 44821146, 44821147, 44821150, 44821151, 44821152, 44821153, 44821154, 44821160, 44821161, 44821166, 44821236, 44821237, 44821238, 44821239, 44821241, 44821242, 44821243, 44821244, 44821245, 44821246, 44821247, 44821248, 44821249, 44821250, 44821251, 44821252, 44821253, 44821254, 44821255, 44821256, 44821557, 44821608, 44821787, 44821824, 44821827, 44821828, 44821949, 44821978, 44821979, 44821987, 44821988, 44822017, 44822018, 44822019, 44822165, 44822166, 44822167, 44822188, 44822189, 44822190, 44822191, 44822208, 44822209, 44822210, 44822211, 44822212, 44822213, 44822216, 44822217, 44822218, 44822219, 44822220, 44822221, 44822227, 44822228, 44822229, 44822235, 44822236, 44822237, 44822289, 44822292, 44822293, 44822294, 44822295, 44822296, 44822297, 44822298, 44822299, 44822300, 44822301, 44822302, 44822303, 44822304, 44822305, 44822306, 44822307, 44822308, 44822309, 44822310, 44822311, 44822312, 44822313, 44822316, 44822668, 44822751, 44822934, 44822935, 44822936, 44822948, 44822953, 44822985, 44822987, 44822988, 44822989, 44823109, 44823144, 44823196, 44823197, 44823333, 44823349, 44823350, 44823360, 44823361, 44823362, 44823367, 44823368, 44823369, 44823370, 44823371, 44823372, 44823373, 44823376, 44823377, 44823378, 44823382, 44823441, 44823442, 44823443, 44823444, 44823445, 44823446, 44823448, 44823449, 44823450, 44823451, 44823452, 44823453, 44823454, 44823455, 44823456, 44823457, 44823458, 44823459, 44823461, 44824071, 44824072, 44824073, 44824074, 44824082, 44824083, 44824092, 44824093, 44824120, 44824121, 44824132, 44824287, 44824288, 44824289, 44824311, 44824312, 44824350, 44824351, 44824352, 44824473, 44824474, 44824475, 44824486, 44824487, 44824510, 44824511, 44824512, 44824513, 44824514, 44824515, 44824519, 44824520, 44824521, 44824522, 44824523, 44824528, 44824536, 44824537, 44824538, 44824540, 44824541, 44824616, 44824617, 44824618, 44824619, 44824620, 44824621, 44824622, 44824623, 44824624, 44824625, 44824626, 44824627, 44824628, 44824629, 44824630, 44824631, 44824632, 44824633, 44824634, 44824635, 44824636, 44825264, 44825280, 44825281, 44825282, 44825298, 44825307, 44825308, 44825499, 44825500, 44825501, 44825551, 44825651, 44825661, 44825677, 44825678, 44825679, 44825680, 44825681, 44825682, 44825696, 44825697, 44825698, 44825699, 44825700, 44825701, 44825702, 44825703, 44825707, 44825708, 44825709, 44825710, 44825711, 44825717, 44825718, 44825719, 44825729, 44825730, 44825731, 44825732, 44825733, 44825734, 44825735, 44825796, 44825801, 44825802, 44825803, 44825804, 44825805, 44825806, 44825807, 44825808, 44825809, 44825810, 44825811, 44825812, 44825813, 44825814, 44825815, 44825816, 44825817, 44825821, 44826155, 44826193, 44826459, 44826460, 44826461, 44826471, 44826503, 44826504, 44826505, 44826506, 44826507, 44826518, 44826519, 44826662, 44826666, 44826678, 44826679, 44826700, 44826701, 44826736, 44826737, 44826848, 44826854, 44826869, 44826870, 44826885, 44826886, 44826887, 44826888, 44826889, 44826890, 44826896, 44826897, 44826902, 44826911, 44826912, 44826913, 44826914, 44826915, 44826969, 44826970, 44826973, 44826974, 44826975, 44826976, 44826977, 44826979, 44826981, 44826983, 44826984, 44827406, 44827615, 44827616, 44827617, 44827629, 44827630, 44827635, 44827661, 44827668, 44827894, 44828003, 44828004, 44828005, 44828019, 44828035, 44828036, 44828037, 44828047, 44828048, 44828049, 44828050, 44828051, 44828052, 44828053, 44828059, 44828060, 44828061, 44828075, 44828076, 44828077, 44828144, 44828145, 44828146, 44828147, 44828148, 44828151, 44828152, 44828153, 44828154, 44828155, 44828156, 44828157, 44828158, 44828159, 44828160, 44828161, 44828162, 44828163, 44828164, 44828165, 44828166, 44828793, 44828794, 44828795, 44828820, 44828833, 44828834, 44828835, 44828841, 44828842, 44829001, 44829012, 44829068, 44829178, 44829179, 44829180, 44829188, 44829195, 44829196, 44829197, 44829213, 44829214, 44829215, 44829216, 44829217, 44829218, 44829219, 44829220, 44829223, 44829225, 44829226, 44829232, 44829233, 44829234, 44829292, 44829293, 44829294, 44829298, 44829299, 44829300, 44829301, 44829302, 44829303, 44829304, 44829305, 44829308, 44829639, 44829878, 44829879, 44829880, 44829881, 44829882, 44829903, 44829926, 44829927, 44829928, 44829936, 44829937, 44829938, 44829939, 44829940, 44829941, 44829942, 44830105, 44830115, 44830137, 44830182, 44830299, 44830310, 44830323, 44830324, 44830325, 44830326, 44830343, 44830345, 44830346, 44830347, 44830348, 44830350, 44830351, 44830352, 44830353, 44830364, 44830365, 44830366, 44830439, 44830440, 44830441, 44830442, 44830443, 44830445, 44830446, 44830447, 44830448, 44830449, 44830450, 44830451, 44830452, 44830453, 44830454, 44830455, 44830457, 44830794, 44830848, 44831045, 44831046, 44831047, 44831059, 44831068, 44831093, 44831094, 44831103, 44831104, 44831105, 44831106, 44831107, 44831108, 44831272, 44831278, 44831279, 44831280, 44831298, 44831299, 44831336, 44831488, 44831489, 44831490, 44831504, 44831505, 44831506, 44831507, 44831508, 44831509, 44831510, 44831512, 44831517, 44831519, 44831524, 44831525, 44831529, 44831583, 44831587, 44831588, 44831589, 44831590, 44831591, 44831592, 44831593, 44831594, 44831595, 44831596, 44831597, 44831598, 44831599, 44831600, 44831601, 44831602, 44831604, 44831605, 44831606, 44832190, 44832191, 44832192, 44832193, 44832194, 44832214, 44832239, 44832240, 44832241, 44832248, 44832249, 44832250, 44832251, 44832423, 44832424, 44832451, 44832602, 44832630, 44832646, 44832647, 44832648, 44832649, 44832650, 44832651, 44832652, 44832653, 44832654, 44832659, 44832663, 44832708, 44832709, 44832710, 44832711, 44832714, 44832715, 44832716, 44832717, 44832718, 44832719, 44832720, 44832722, 44832723, 44833100, 44833101, 44833365, 44833366, 44833367, 44833368, 44833387, 44833394, 44833403, 44833414, 44833415, 44833421, 44833556, 44833611, 44833628, 44833629, 44833630, 44833631, 44833632, 44833670, 44833787, 44833810, 44833811, 44833812, 44833813, 44833831, 44833832, 44833833, 44833834, 44833835, 44833836, 44833837, 44833838, 44833847, 44833848, 44833849, 44833850, 44833851, 44833865, 44833866, 44833912, 44833913, 44833914, 44833917, 44833918, 44833919, 44833920, 44833921, 44833922, 44833923, 44833924, 44833925, 44834342, 44834548, 44834549, 44834563, 44834564, 44834565, 44834569, 44834600, 44834601, 44834602, 44834603, 44834715, 44834764, 44834769, 44834797, 44834841, 44834842, 44834952, 44834953, 44834975, 44834976, 44834977, 44834978, 44834979, 44834980, 44834998, 44834999, 44835000, 44835001, 44835002, 44835003, 44835006, 44835007, 44835008, 44835009, 44835015, 44835022, 44835023, 44835081, 44835082, 44835083, 44835084, 44835085, 44835086, 44835087, 44835088, 44835089, 44835090, 44835093, 44835474, 44835758, 44835769, 44835791, 44835792, 44835974, 44836009, 44836041, 44836042, 44836149, 44836150, 44836159, 44836171, 44836172, 44836173, 44836174, 44836175, 44836189, 44836190, 44836191, 44836192, 44836193, 44836194, 44836195, 44836202, 44836203, 44836204, 44836205, 44836214, 44836215, 44836223, 44836224, 44836225, 44836226, 44836287, 44836288, 44836289, 44836293, 44836294, 44836295, 44836296, 44836297, 44836298, 44836302, 44836303, 44836914, 44836915, 44836916, 44836917, 44836918, 44836929, 44836944, 44836945, 44836970, 44836971, 44836972, 44836973, 44836984, 44836985, 44837129, 44837135, 44837136, 44837309, 44837310, 44837317, 44837335, 44837347, 44837348, 44837349, 44837350, 44837351, 44837352, 44837353, 44837354, 44837358, 44837359, 44837360, 44837361, 44837362, 44837363, 44837369, 44837375, 44837376, 44837423, 44837424, 44837425, 44837430, 44837433, 44837434, 44837435, 44837436, 44837437, 44837438, 44837439, 44837440, 44837441, 44837442, 44837443, 44837444, 44837445, 44837446, 44837449, 44837823, 45533019, 45533020, 45533021, 45533022, 45533023, 45533085, 45533136, 45533137, 45533138, 45533140, 45533147, 45533148, 45533149, 45533277, 45533353, 45533354, 45533439, 45533746, 45533747, 45533748, 45533749, 45533807, 45533808, 45533810, 45533811, 45533812, 45533813, 45533814, 45533815, 45533816, 45533817, 45533818, 45533819, 45533848, 45533849, 45533850, 45533851, 45533852, 45533853, 45533867, 45533868, 45533881, 45533883, 45533884, 45533885, 45533886, 45533887, 45533939, 45533940, 45533941, 45533942, 45534020, 45534021, 45534023, 45534024, 45534025, 45534026, 45534027, 45534130, 45534422, 45534424, 45534427, 45534428, 45534429, 45534430, 45534433, 45534434, 45534435, 45534440, 45534458, 45534479, 45537591, 45537600, 45537698, 45537961, 45537962, 45538046, 45538069, 45538108, 45538109, 45538110, 45538111, 45538112, 45538114, 45538115, 45538121, 45538122, 45538123, 45538143, 45538241, 45538312, 45538313, 45538314, 45538315, 45538373, 45538374, 45538375, 45538377, 45538586, 45538681, 45538682, 45538683, 45538687, 45538688, 45538689, 45538735, 45538737, 45538738, 45538739, 45538740, 45538742, 45538743, 45538744, 45538745, 45538746, 45538747, 45538748, 45538773, 45538774, 45538788, 45538809, 45538814, 45538815, 45538816, 45538817, 45538818, 45538819, 45538866, 45538867, 45538868, 45538944, 45538949, 45538950, 45538951, 45538952, 45539314, 45539371, 45542487, 45542738, 45542780, 45542781, 45542821, 45542887, 45542893, 45542911, 45542912, 45543167, 45543168, 45543269, 45543270, 45543322, 45543324, 45543485, 45543486, 45543487, 45543547, 45543548, 45543549, 45543551, 45543553, 45543554, 45543555, 45543556, 45543557, 45543558, 45543560, 45543561, 45543590, 45543592, 45543593, 45543594, 45543612, 45543613, 45543631, 45543633, 45543634, 45543635, 45543636, 45543689, 45543690, 45543772, 45543773, 45543774, 45543775, 45544130, 45544133, 45544134, 45544169, 45547372, 45547625, 45547626, 45547627, 45547657, 45547704, 45547707, 45547708, 45547732, 45547734, 45547735, 45547743, 45547744, 45547763, 45547840, 45547841, 45547842, 45547843, 45547933, 45547936, 45547937, 45547938, 45548010, 45548011, 45548012, 45548013, 45548116, 45548117, 45548163, 45548321, 45548323, 45548324, 45548325, 45548326, 45548384, 45548385, 45548386, 45548387, 45548388, 45548392, 45548393, 45548394, 45548395, 45548396, 45548397, 45548423, 45548424, 45548443, 45548444, 45548465, 45548466, 45548503, 45548569, 45548570, 45548571, 45548575, 45548576, 45548944, 45548948, 45548951, 45548952, 45548953, 45548954, 45548996, 45552047, 45552385, 45552386, 45552417, 45552453, 45552454, 45552527, 45552528, 45552530, 45552537, 45552538, 45552539, 45552551, 45552638, 45552639, 45552720, 45552721, 45552885, 45552937, 45552938, 45552939, 45552940, 45552941, 45553098, 45553099, 45553148, 45553149, 45553151, 45553152, 45553153, 45553154, 45553155, 45553156, 45553157, 45553158, 45553159, 45553160, 45553161, 45553162, 45553163, 45553189, 45553204, 45553206, 45553224, 45553225, 45553226, 45553227, 45553271, 45553352, 45553353, 45553355, 45553356, 45553357, 45553714, 45553715, 45553716, 45553717, 45553718, 45553719, 45553748, 45556807, 45557112, 45557113, 45557236, 45557237, 45557238, 45557240, 45557376, 45557377, 45557378, 45557379, 45557464, 45557465, 45557538, 45557539, 45557626, 45557661, 45557662, 45557664, 45557807, 45557808, 45557809, 45557850, 45557851, 45557852, 45557853, 45557854, 45557855, 45557856, 45557857, 45557858, 45557859, 45557884, 45557885, 45557886, 45557901, 45557902, 45557903, 45557925, 45557926, 45557927, 45557928, 45557929, 45557992, 45557993, 45557994, 45558062, 45558063, 45558064, 45558066, 45558069, 45558453, 45558454, 45558455, 45558456, 45558459, 45558483, 45558493, 45561616, 45561949, 45561985, 45561986, 45562025, 45562087, 45562088, 45562093, 45562095, 45562096, 45562109, 45562204, 45562206, 45562207, 45562271, 45562272, 45562273, 45562274, 45562343, 45562344, 45562457, 45562650, 45562651, 45562691, 45562692, 45562693, 45562694, 45562696, 45562697, 45562698, 45562699, 45562700, 45562701, 45562732, 45562733, 45562769, 45562771, 45562772, 45562773, 45562774, 45562775, 45562776, 45562777, 45562778, 45562837, 45562838, 45562839, 45562840, 45562926, 45562927, 45562930, 45562933, 45562934, 45562935, 45562936, 45562937, 45563013, 45563288, 45563290, 45563299, 45563315, 45566384, 45566731, 45566810, 45566874, 45566875, 45566876, 45566884, 45566885, 45566886, 45566887, 45566906, 45566907, 45567013, 45567093, 45567094, 45567095, 45567097, 45567167, 45567168, 45567265, 45567266, 45567315, 45567317, 45567469, 45567472, 45567473, 45567474, 45567524, 45567525, 45567526, 45567528, 45567529, 45567530, 45567531, 45567532, 45567533, 45567534, 45567535, 45567564, 45567578, 45567579, 45567580, 45567603, 45567605, 45567608, 45567610, 45567611, 45567659, 45567660, 45567743, 45567746, 45567748, 45567749, 45567750, 45567751, 45567752, 45567753, 45567754, 45567842, 45568112, 45568114, 45568115, 45568118, 45568146, 45568157, 45571289, 45571314, 45571329, 45571741, 45571742, 45571743, 45571802, 45571803, 45571804, 45571805, 45571810, 45571811, 45571812, 45572168, 45572169, 45572170, 45572171, 45572217, 45572219, 45572374, 45572375, 45572416, 45572417, 45572418, 45572419, 45572420, 45572421, 45572422, 45572423, 45572424, 45572425, 45572426, 45572427, 45572428, 45572429, 45572430, 45572431, 45572432, 45572433, 45572434, 45572435, 45572436, 45572437, 45572438, 45572439, 45572458, 45572459, 45572460, 45572461, 45572471, 45572492, 45572494, 45572495, 45572496, 45572497, 45572498, 45572552, 45572627, 45572630, 45572635, 45572636, 45572637, 45572638, 45572639, 45573007, 45573009, 45573010, 45573011, 45573012, 45573016, 45573032, 45573037, 45576443, 45576480, 45576521, 45576541, 45576545, 45576546, 45576547, 45576590, 45576591, 45576592, 45576593, 45576700, 45576787, 45576788, 45576866, 45576868, 45576951, 45576952, 45576994, 45577165, 45577168, 45577217, 45577219, 45577221, 45577222, 45577223, 45577224, 45577225, 45577226, 45577227, 45577228, 45577229, 45577230, 45577231, 45577232, 45577233, 45577234, 45577235, 45577268, 45577269, 45577306, 45577307, 45577310, 45577313, 45577314, 45577315, 45577358, 45577359, 45577435, 45577437, 45577776, 45577778, 45577781, 45577782, 45577783, 45577786, 45577787, 45577807, 45577808, 45577809, 45577823, 45577828, 45577829, 45581352, 45581353, 45581354, 45581355, 45581435, 45581461, 45581491, 45581492, 45581497, 45581498, 45581499, 45581500, 45581519, 45581626, 45581627, 45581702, 45581703, 45581766, 45581853, 45581860, 45581904, 45582061, 45582062, 45582063, 45582064, 45582108, 45582109, 45582110, 45582111, 45582112, 45582113, 45582114, 45582115, 45582116, 45582117, 45582118, 45582120, 45582121, 45582122, 45582156, 45582157, 45582174, 45582175, 45582176, 45582197, 45582201, 45582202, 45582247, 45582248, 45582249, 45582250, 45582251, 45582316, 45582318, 45582319, 45582694, 45582695, 45582701, 45582718, 45582732, 45582734, 45582735, 45585781, 45585793, 45585857, 45586119, 45586139, 45586140, 45586173, 45586174, 45586219, 45586220, 45586243, 45586289, 45586290, 45586291, 45586292, 45586297, 45586298, 45586299, 45586421, 45586506, 45586572, 45586573, 45586574, 45586575, 45586674, 45586675, 45586725, 45586726, 45586891, 45586892, 45586893, 45586894, 45586936, 45586937, 45586938, 45586939, 45586940, 45586942, 45586943, 45586944, 45586945, 45586946, 45586947, 45586948, 45586975, 45586993, 45587008, 45587013, 45587014, 45587015, 45587016, 45587018, 45587064, 45587065, 45587066, 45587067, 45587068, 45587069, 45587157, 45587160, 45587161, 45587162, 45587163, 45587496, 45587497, 45587523, 45587538, 45590768, 45591027, 45591029, 45591030, 45591031, 45591051, 45591115, 45591116, 45591178, 45591181, 45591185, 45591186, 45591201, 45591297, 45591298, 45591299, 45591300, 45591388, 45591456, 45591458, 45591459, 45591460, 45591461, 45591555, 45591558, 45591559, 45591596, 45591597, 45591598, 45591599, 45591743, 45591744, 45591745, 45591746, 45591747, 45591800, 45591802, 45591804, 45591805, 45591806, 45591807, 45591808, 45591809, 45591810, 45591811, 45591812, 45591846, 45591847, 45591848, 45591851, 45591852, 45591853, 45591885, 45591888, 45591889, 45591890, 45591934, 45591935, 45591936, 45592028, 45592030, 45592034, 45592133, 45592134, 45592135, 45592407, 45592429, 45592434, 45592442, 45592444, 45595569, 45595797, 45595798, 45595799, 45595884, 45595885, 45595940, 45595942, 45595947, 45596049, 45596129, 45596130, 45596131, 45596197, 45596198, 45596199, 45596335, 45596388, 45596495, 45596497, 45596498, 45596546, 45596548, 45596549, 45596550, 45596551, 45596552, 45596553, 45596554, 45596592, 45596593, 45596594, 45596595, 45596630, 45596752, 45596757, 45596762, 45596763, 45596764, 45596766, 45596767, 45596768, 45596769, 45596770, 45596771, 45596864, 45597165, 45597167, 45597168, 45597169, 45597170, 45597171, 45597192, 45597197, 45597199, 45597211, 45597212, 45600374, 45600375, 45600609, 45600641, 45600642, 45600659, 45600719, 45600720, 45600746, 45600747, 45600785, 45600786, 45600793, 45600888, 45600889, 45600958, 45601024, 45601026, 45601027, 45601028, 45601125, 45601133, 45601134, 45601182, 45601354, 45601356, 45601416, 45601418, 45601419, 45601421, 45601422, 45601423, 45601424, 45601425, 45601479, 45601480, 45601495, 45601496, 45601497, 45601498, 45601499, 45601545, 45601546, 45601547, 45601623, 45601625, 45601626, 45601627, 45601629, 45601630, 45601631, 45601633, 45601634, 45601635, 45601727, 45602002, 45602003, 45602004, 45602005, 45602006, 45602007, 45602008, 45602011, 45602032, 45602041, 45605095, 45605401, 45605402, 45605403, 45605404, 45605405, 45605476, 45605543, 45605545, 45605547, 45605556, 45605558, 45605559, 45605560, 45605643, 45605645, 45605784, 45605785, 45605786, 45605787, 45605788, 45605943, 45605944, 45605946, 45606131, 45606132, 45606133, 45606191, 45606192, 45606193, 45606194, 45606195, 45606196, 45606197, 45606198, 45606199, 45606200, 45606201, 45606202, 45606203, 45606204, 45606205, 45606206, 45606227, 45606229, 45606270, 45606272, 45606273, 45606274, 45606276, 45606323, 45606324, 45606411, 45606412, 45606413, 45606414, 45606793, 45606794, 45606795, 45606796, 45606797, 45606799, 45606832, 45606833, 45609901, 713855, 725363, 725364, 725365, 725366, 725378, 725379, 725380, 725388, 725389, 725390, 725458, 725459, 766349, 766412, 766413, 766414, 766424, 766425, 766426, 766427, 766428, 766429, 766430, 766431, 766437
                                            ) 
                                            AND full_text LIKE '%_rank1]%'
                                    ) a 
                                        ON (
                                            c.path LIKE CONCAT('%.',
                                        a.id,
                                        '.%') 
                                        OR c.path LIKE CONCAT('%.',
                                        a.id) 
                                        OR c.path LIKE CONCAT(a.id,
                                        '.%') 
                                        OR c.path = a.id) 
                                    WHERE
                                        is_standard = 0 
                                        AND is_selectable = 1
                                    )
                            )
                        ) c_occurrence 
                    LEFT JOIN
                        `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_standard_concept 
                            ON c_occurrence.condition_concept_id = c_standard_concept.concept_id 
                    LEFT JOIN
                        `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_type 
                            ON c_occurrence.condition_type_concept_id = c_type.concept_id 
                    LEFT JOIN
                        `""" + os.environ["WORKSPACE_CDR"] + """.visit_occurrence` v 
                            ON c_occurrence.visit_occurrence_id = v.visit_occurrence_id 
                    LEFT JOIN
                        `""" + os.environ["WORKSPACE_CDR"] + """.concept` visit 
                            ON v.visit_concept_id = visit.concept_id 
                    LEFT JOIN
                        `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_source_concept 
                            ON c_occurrence.condition_source_concept_id = c_source_concept.concept_id 
                    LEFT JOIN
                        `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_status 
                            ON c_occurrence.condition_status_concept_id = c_status.concept_id"""

    person_information_all = pd.read_gbq(
        dataset_25030714_condition_sql,
        dialect="standard",
        use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
        progress_bar_type="tqdm_notebook")
    
    # Keep only the information we want
    person_information = person_information_all[['person_id', 'standard_concept_name', 'condition_start_datetime',
                                               'condition_end_datetime', 'source_concept_name', 'source_concept_code',
                                               'source_vocabulary']]

    # Filter for ICD10CM and ICD9CM source vocabularies
    person_information_all = person_information_all[person_information_all['source_vocabulary'].isin(['ICD10CM', 'ICD9CM'])]

    # Prepare message
    message = "Read the personal information about the patient and kept only data with ICD-10 and ICD-9."

    return person_information_all, message
def read_person_information_df():
    """
    Reads person information from BigQuery and filters it to keep only data with 'Male' and 'Female' sexes.

    Returns:
    pd.DataFrame: DataFrame containing filtered person information.
    str: Message indicating the action taken.
    """
    # This query represents dataset "All participants" for domain "person" and was generated for All of Us Controlled Tier Dataset v7
    dataset_25030714_person_sql = """
        SELECT
            person.person_id,
            person.gender_concept_id,
            p_gender_concept.concept_name as gender,
            person.birth_datetime as date_of_birth,
            person.race_concept_id,
            p_race_concept.concept_name as race,
            person.ethnicity_concept_id,
            p_ethnicity_concept.concept_name as ethnicity,
            person.sex_at_birth_concept_id,
            p_sex_at_birth_concept.concept_name as sex_at_birth
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.person` person
        LEFT JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_gender_concept 
                ON person.gender_concept_id = p_gender_concept.concept_id 
        LEFT JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_race_concept 
                ON person.race_concept_id = p_race_concept.concept_id 
        LEFT JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_ethnicity_concept 
                ON person.ethnicity_concept_id = p_ethnicity_concept.concept_id 
        LEFT JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_sex_at_birth_concept 
                ON person.sex_at_birth_concept_id = p_sex_at_birth_concept.concept_id"""

    person_df = pd.read_gbq(
        dataset_25030714_person_sql,
        dialect="standard",
        use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
        progress_bar_type="tqdm_notebook")
    
    # Keep only the information we want
    person_df_all = person_df[['person_id', 'date_of_birth', 'race', 'ethnicity', 'sex_at_birth']]

    # Filter for 'Male' and 'Female' sexes
    person_df_all = person_df_all[person_df_all['sex_at_birth'].isin(['Male', 'Female'])]

    # Prepare message
    message = "Read the personal information about the patient and kept only data with 'Male' and 'Female' sexes."

    return person_df_all, message
def copy_files_to_local(bucket):
    """
    Copies files from Google Cloud Storage to the local directory './data' if they don't already exist.

    Args:
    bucket (str): The Google Cloud Storage bucket where the files are located.

    Returns:
    None
    """
    # Check if the folders exist at the specified path './'
    folders = ['data', 'new_run', 'download']
    for folder in folders:
        folder_path = os.path.join('.', folder)
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
            print(f"Created folder: {folder}")

    # Define the paths for the files
    files = {
        'ancestry_preds.tsv': 'gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/aux/ancestry/ancestry_preds.tsv',
        'relatedness.tsv': 'gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/aux/relatedness/relatedness.tsv',
        'relatedness_flagged_samples.tsv': 'gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/aux/relatedness/relatedness_flagged_samples.tsv'
    }

    # Copy files using gsutil
    for file_name, file_path in files.items():
        local_file_path = os.path.join('.', 'data', file_name)
        if not os.path.exists(local_file_path):
            gsutil_command = f"gsutil -u $GOOGLE_PROJECT cp {file_path} {bucket}/data/"
            print(f"Copying file {file_name} to {bucket}/data/ ...")
            !{gsutil_command}
            if os.path.exists(local_file_path):
                print(f"File {file_name} copied successfully.")
            else:
                print(f"Failed to copy file {file_name} you have the file there already.")
def read_zip_code_socioeconomic_data():
    """
    Reads zip code socioeconomic data from BigQuery.

    Returns:
    pd.DataFrame: DataFrame containing zip code socioeconomic data.
    str: Message indicating the action taken.
    """
    # This query represents dataset "test2" for domain "zip_code_socioeconomic" and was generated for All of Us Controlled Tier Dataset v7
    dataset_65271084_zip_code_socioeconomic_sql = """
        SELECT
            observation.person_id,
            observation.observation_datetime,
            zip_code.zip3_as_string as zip_code,
            zip_code.fraction_assisted_income as assisted_income,
            zip_code.fraction_high_school_edu as high_school_education,
            zip_code.median_income,
            zip_code.fraction_no_health_ins as no_health_insurance,
            zip_code.fraction_poverty as poverty,
            zip_code.fraction_vacant_housing as vacant_housing,
            zip_code.deprivation_index,
            zip_code.acs as american_community_survey_year 
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.zip3_ses_map` zip_code 
        JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.observation` observation 
                ON CAST(SUBSTR(observation.value_as_string,
            0,
            STRPOS(observation.value_as_string,
            '*') - 1) AS INT64) = zip_code.zip3 
            AND observation_source_concept_id = 1585250 
            AND observation.value_as_string NOT LIKE 'Res%'"""

    zip_code_socioeconomic_df = pd.read_gbq(
        dataset_65271084_zip_code_socioeconomic_sql,
        dialect="standard",
        use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
        progress_bar_type="tqdm_notebook")
    
    message = "Read zip code socioeconomic data successfully."

    return zip_code_socioeconomic_df, message
def run_analysis():
    """
    Runs the analysis, resets the Person_df and Person_information, and returns the necessary dataframes along with messages.

    Returns:
    pd.DataFrame: DataFrame containing Person_df after resetting.
    pd.DataFrame: DataFrame containing Person_information after resetting.
    pd.Series: Series containing counts of individuals in each race.
    pd.Series: Series containing counts of Hispanic individuals in each race.
    pd.DataFrame: DataFrame containing Population data.
    str: Message indicating the action taken.
    """

    # reset Person_df and Person_information
    person_df = person_df_all.copy()
    person_information = person_information_all.copy()

    # Count the number of individuals in each race
    race_counts = person_df['race'].value_counts()
    race_counts = race_counts.apply(lambda x: '<20' if x < 20 else x)
    race_counts_message = "Number of individuals in each race:\n" + str(race_counts)

    # Count the number of Hispanic individuals in each race
    hispanic_counts = person_df[person_df['ethnicity'] == 'Hispanic or Latino']['race'].value_counts()
    hispanic_counts = hispanic_counts.apply(lambda x: '<20' if x < 20 else x)
    hispanic_counts_message = "Number of Hispanic individuals in each race:\n" + str(hispanic_counts)

    # Prepare Population data
    population = read_tsv('ancestry_preds.tsv')
    population_message = "\nPopulation data loaded successfully."

    # Convert string representation of lists to actual lists
    population['pca_features'] = population['pca_features'].apply(ast.literal_eval)
    population['probabilities'] = population['probabilities'].apply(ast.literal_eval)

    # Extracting PCA features into separate columns
    pca_columns = [f'principal_component_{i}' for i in range(1, 17)]
    population[pca_columns] = pd.DataFrame(population['pca_features'].tolist(), index=population.index)

    # Extracting probabilities into separate columns
    probability_columns = [f'probability_{i}' for i in range(1, 7)]
    population[probability_columns] = pd.DataFrame(population['probabilities'].tolist(), index=population.index)

    # Create a new column 'max_probability' with the maximum probability across columns
    population['max_probability'] = population[probability_columns].max(axis=1)

    max_probability_message = "Maximum probability calculated successfully."

    return person_df, person_information, race_counts, hispanic_counts, population, race_counts_message, hispanic_counts_message, population_message, max_probability_message
def plot_pca_graphs(population):
    """
    Generates PCA scatterplots and a line chart based on ancestry prediction and saves them as images.

    Args:
    population (pd.DataFrame): DataFrame containing population data.

    Returns:
    str: Message indicating the action taken.
    """

    # Scatterplot for 'ancestry_pred_other'
    plt.figure(figsize=(10, 6))
    colors = {'eur': 'darkblue', 'afr': 'darkred', 'amr': 'darkgreen', 'oth': 'gold', 'eas': 'purple', 'sas': 'darkorange', 'mid': 'black'}

    for ancestry, color in colors.items():
        subset = population[population['ancestry_pred_other'].eq(ancestry)]
        plt.scatter(subset['principal_component_1'], subset['principal_component_2'], label=ancestry, color=color, alpha=0.7, s=3)

    plt.title('PCA Scatterplot - Ancestry_pred_other')
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    plt.legend()
    plt.savefig('./download/PCA_Scatterplot_Ancestry_pred_other.png')
    plt.close()

    # Scatterplot for 'ancestry_pred'
    plt.figure(figsize=(10, 6))
    colors = {'eur': 'darkblue', 'afr': 'darkred', 'amr': 'darkgreen', 'eas': 'purple', 'sas': 'darkorange', 'mid': 'black'}

    for ancestry, color in colors.items():
        subset = population[population['ancestry_pred'].eq(ancestry)]
        plt.scatter(subset['principal_component_1'], subset['principal_component_2'], label=ancestry, color=color, alpha=0.5, s=3)

    plt.title('PCA Scatterplot - Ancestry_pred')
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    plt.legend()
    plt.savefig('./download/PCA_Scatterplot_Ancestry_pred.png')
    plt.close()

    # Line Chart
    principal_components = ['principal_component_{}'.format(i) for i in range(1, 17)]
    colors = {'oth':'gold','eur': 'darkblue', 'afr': 'darkred', 'amr': 'darkgreen', 'eas': 'purple', 'sas': 'darkorange', 'mid': 'black'}

    plt.figure(figsize=(12, 8))
    for ancestry, color in colors.items():
        mean_values = population.loc[population['ancestry_pred_other'] == ancestry, principal_components].median()
        plt.plot(principal_components, mean_values, label=ancestry, color=color, marker='o',alpha=0.5)

    plt.xticks(rotation=90)
    plt.xticks(ticks=range(0, 16), labels=['PCA{}'.format(i) for i in range(1, 17)])
    plt.title('Line Chart - Principal Components by Ancestry Prediction')
    plt.xlabel('Principal Components')
    plt.ylabel('Median Values')
    plt.legend()
    plt.savefig('./download/Line_Chart_Principal_Components_by_Ancestry_Prediction.png')
    plt.close()

    return "PCA graphs saved successfully."
def perform_pca(population_data):
    """
    Performs PCA (Principal Component Analysis) on the population data and creates scatterplots and a line chart.

    Args:
    population_data (pd.DataFrame): DataFrame containing population data.

    Returns:
    PCA: PCA object containing the transformed data.
    str: Message indicating the action taken.
    """
    # Selecting features for PCA
    pca_features = population_data.iloc[:, 8:24]

    # Initialize PCA
    pca = PCA(n_components=2)

    # Fit and transform the data
    pca_data = pca.fit_transform(pca_features)

    # Create scatterplot for 'ancestry_pred_other'
    plt.figure(figsize=(10, 6))
    colors = {'eur': 'darkblue', 'afr': 'darkred', 'amr': 'darkgreen', 'oth': 'gold', 'eas': 'purple', 'sas': 'darkorange', 'mid': 'black'}

    for ancestry, color in colors.items():
        subset = population_data[population_data['ancestry_pred_other'].eq(ancestry)]
        plt.scatter(pca_data[subset.index, 0], pca_data[subset.index, 1], label=ancestry, color=color, alpha=0.7, s=3)

    plt.title('PCA Scatterplot - Ancestry_pred_other')
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    plt.legend()
    plt.savefig('./download/PCA_Scatterplot_Ancestry_pred_other_1.png')
    plt.show()

    # Create scatterplot for 'ancestry_pred'
    plt.figure(figsize=(10, 6))
    colors = {'eur': 'darkblue', 'afr': 'darkred', 'amr': 'darkgreen', 'eas': 'purple', 'sas': 'darkorange', 'mid': 'black'}

    for ancestry, color in colors.items():
        subset = population_data[population_data['ancestry_pred'].eq(ancestry)]
        plt.scatter(pca_data[subset.index, 0], pca_data[subset.index, 1], label=ancestry, color=color, alpha=0.7, s=3)

    plt.title('PCA Scatterplot - Ancestry_pred')
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    plt.legend()
    plt.savefig('./download/PCA_Scatterplot_Ancestry_pred_1.png')
    plt.show()

    # Create line chart
    plt.figure(figsize=(12, 8))

    # List of principal components
    principal_components = ['principal_component_{}'.format(i) for i in range(1, 17)]

    # Set the color palette for each ancestry prediction
    colors = {'oth':'gold','eur': 'darkblue', 'afr': 'darkred', 'amr': 'darkgreen', 'eas': 'purple', 'sas': 'darkorange', 'mid': 'black'}

    # Loop through each ancestry prediction and plot a line for each
    for ancestry, color in colors.items():
        # Calculate the mean of each principal component for the current ancestry
        mean_values = population_data.loc[population_data['ancestry_pred_other'] == ancestry, principal_components].median()
        plt.plot(principal_components, mean_values, label=ancestry, color=color, marker='o')

    # Rotate x-axis labels
    plt.xticks(rotation=90)

    # Rename x-axis labels
    plt.xticks(ticks=range(0, 16), labels=['PCA{}'.format(i) for i in range(1, 17)])

    plt.title('Line Chart - Principal Components by Ancestry Prediction')
    plt.xlabel('Principal Components')
    plt.ylabel('Median Values')
    plt.legend()
    plt.show()

    return pca, "PCA performed successfully. Scatterplots and line chart created."
def prepare_genotyping_data(pop_variable, population, person_df, person_information):
    """
    Prepares data for genotyping analysis based on the specified population variable.

    Args:
    pop_variable (str): The population variable to filter the data.
    population (pd.DataFrame): DataFrame containing population data.
    person_df (pd.DataFrame): DataFrame containing person data.
    person_information (pd.DataFrame): DataFrame containing person information data.

    Returns:
    pd.DataFrame: DataFrame containing filtered population data.
    pd.DataFrame: DataFrame containing filtered person data.
    pd.DataFrame: DataFrame containing filtered person information data.
    str: Message indicating the action taken.
    """
    if pop_variable == 'all':
        message = "Using all Ancestries for genotyping analysis."
    else:
        population = population[population['ancestry_pred'] == pop_variable]
        message = f"Using {pop_variable} population ancestry for genotyping analysis."

    population.rename(columns={'research_id': 'person_id'}, inplace=True)
    ids = population['person_id'].astype(str).tolist()

    person_df['person_id'] = person_df['person_id'].astype(str)
    person_df = person_df[person_df['person_id'].isin(ids)].reset_index(drop=True)

    person_information['person_id'] = person_information['person_id'].astype(str)
    person_information = person_information[person_information['person_id'].isin(ids)].reset_index(drop=True)

    return population, person_df, person_information, message
def remove_related_samples(person_df):
    """
    Removes samples that are flagged as related from the person DataFrame.

    Args:
    person_df (pd.DataFrame): DataFrame containing person data.
    relatedness_flaged_samples (pd.DataFrame): DataFrame containing relatedness flagged samples.

    Returns:
    pd.DataFrame: DataFrame containing person data with related samples removed.
    str: Message indicating the action taken.
    """
    relatedness = read_tsv('relatedness.tsv')
    relatedness_flaged_samples = read_tsv('relatedness_flagged_samples.tsv')
    ids = relatedness_flaged_samples['sample_id'].astype(str).tolist()
    person_df = person_df[~person_df['person_id'].isin(ids)].reset_index(drop=True)
    message = f"{len(ids)} samples are flagged as related and removed from the person DataFrame."
    return person_df, message
def transform_genotypes(df, alleles_row, reference_allele=None, alternative_allele=None):
    # Step 1: Initialize the message
    message = ""

    # Step 2: Remove rows with missing genotype information
    df = df.dropna(subset=['GT'])

    # Step 4: Convert 'GT' column to string type
    df['GT'] = df['GT'].astype(str)

    # Step 4: Split 'GT' column into separate alleles
    df[['Allele_1', 'Allele_2']] = df['GT'].str.split('/', expand=True)

    # Step 6: Set the reference allele
    if reference_allele is None:
        if len(alleles_row) == 2:
            reference_allele = alleles_row[0]
            alternative_allele = alleles_row[1]
        else:
            reference_allele = alleles_row[0]

    # Step 8: Map allele values to letters
    allele_mapping = {str(i): allele for i, allele in enumerate(alleles_row)}
    df['Allele_1_map'] = df['Allele_1'].map(allele_mapping)
    df['Allele_2_map'] = df['Allele_2'].map(allele_mapping)

    # Step 9: Create 'Allele_combination' column and count occurrences
    df['Allele_combination'] = df['Allele_1_map'] + '/' + df['Allele_2_map']
    allele_counts = df['Allele_combination'].value_counts().reset_index()
    allele_counts.columns = ['Allele_combination', 'Allele_count']

    # Step 10: Determine alternative allele
    if len(alleles_row) != 2:
        alternative_combination = ""
        if alternative_allele is None:
            # Step 11: Find all combinations except reference/reference
            alternative_combinations = allele_counts[~allele_counts['Allele_combination'].str.contains(reference_allele)]
            if not alternative_combinations.empty:
                # Step 13: Find the most frequent combination with reference allele
                max_combination = alternative_combinations.loc[alternative_combinations['Allele_count'].idxmax()]
                alternative_combination = max_combination['Allele_combination']
                # Step 14: Set alternative allele as the non-reference allele in the most frequent combination
                alternative_allele = alternative_combination.replace(reference_allele, '')
    else:
        # If alleles_row has length 2, set the second allele as the alternative allele
        alternative_allele = alleles_row[1]

    # At this point, we have the reference and alternative alleles
    # Step 12 Remove reference allele from alternative_combination
    if len(alleles_row) != 2:
        if alternative_combination:
            alternative_allele = alternative_combination.replace(reference_allele, '')

    # Step 13 Create 'GT' column
    df['GT'] = -1  # Initialize all to -1

    # handle combination alternative allele
    if alternative_allele is None:
        alternative_combination = ""
        alternative_combinations = allele_counts[~allele_counts['Allele_combination'].str.contains(reference_allele)]
        if not alternative_combinations.empty:
            max_combination = alternative_combinations.loc[alternative_combinations['Allele_count'].idxmax()]
            alternative_combination = max_combination['Allele_combination']
            alternative_allele = alternative_combination.replace(reference_allele, '')

    # Step 14 Split the alternative allele if it's a combination
    if alternative_allele and '/' in alternative_allele:
        alternative_alleles = alternative_allele.split('/')
    else:
        alternative_alleles = [alternative_allele] if alternative_allele else []

    # Step 15 Assign genotype values based on reference and alternative alleles
    df.loc[(df['Allele_1_map'] == reference_allele) & (df['Allele_2_map'] == reference_allele), 'GT'] = 0
    if alternative_alleles:
        df.loc[(df['Allele_1_map'] == reference_allele) & (df['Allele_2_map'] == alternative_alleles[0]), 'GT'] = 1
        df.loc[(df['Allele_1_map'] == alternative_alleles[0]) & (df['Allele_2_map'] == reference_allele), 'GT'] = 1

    # Check if genotype 1 is present but genotype 2 is not
    if 1 in df['GT'].values and 2 not in df['GT'].values:
        # Set genotype 1 where Allele_1_map is the first allele in alternative_alleles
        df.loc[(df['Allele_1_map'] == alternative_alleles[0]) & (df['Allele_2_map'] == alternative_alleles[0]), 'GT'] = 1

    # Step 16 Check if alternative_alleles contains two or more elements
    if len(alternative_alleles) >=1:
        df.loc[(df['Allele_1_map'] == alternative_alleles[0]) & (df['Allele_2_map'] == alternative_alleles[0]), 'GT'] = 2

    # Step 17 Remove rows with GT == -1 and keep only the reference and alternative alleles
    df = df[df['GT'] != -1]
    
    # Filter allele combinations with counts less than 20
    filtered_allele_counts = allele_counts[allele_counts['Allele_count'] >= 20]

    
    # Step 18: Generate message
    message += "Alleles for SNP: {}\n".format(alleles_row)
    message += "Reference allele: {}\n".format(reference_allele)
    message += "Alternative allele: {}\n".format(alternative_allele)
    message += "Allele combinations count:\n{}\n".format(allele_counts)
    message += "Removed {} rows with missing genotype information.\n".format(len(df[df['GT'].isna()]))

    return df, message
def create_pie_chart(genotype_counts, rs_name, pop_variable):
    """
    Create a pie chart based on genotype counts.

    Args:
    genotype_counts (pd.Series): Series containing genotype counts.
    rs_name (str): Name of the SNP.
    pop_variable (str): Name of the population variable.

    Returns:
    None
    """
    # Replace values less than 20 with a placeholder
    genotype_counts = genotype_counts.apply(lambda x: x if x >= 20 else '<20')
    if 1 not in genotype_counts.index:
        # If '2' is not present, set its count to 0
        genotype_counts[1] = 0
    # Check if genotype '2' is present in the counts
    if 2 not in genotype_counts.index:
        # If '2' is not present, set its count to 0
        genotype_counts[2] = 0
    

    # Plot the pie chart
    pie_chart = genotype_counts.plot(kind='pie', title='{}'.format(rs_name), legend=True)
    
    
    # Add the genotype counts to the title
    title_with_counts = '{}  with {} population(0: {}, 1: {}, 2: {})'.format(rs_name,pop_variable, 
                                                         genotype_counts.get(0, 0),
                                                         genotype_counts.get(1, 0),
                                                         genotype_counts.get(2, 0))
    pie_chart.set_title(title_with_counts)

    # Save the figure
    plt.savefig('./download/{}_{}_pie_GT_my_data_before_remove_data.png'.format(rs_name, pop_variable), dpi=300, facecolor='w',bbox_inches='tight')

    # Show the plot
    plt.show()
def filter_by_ethnicity(person_df, ethnicity=None):
    """
    Filters the person DataFrame based on ethnicity.

    Args:
    person_df (pd.DataFrame): DataFrame containing person data.
    ethnicity (str or None): The ethnicity to filter by. If None, no filtering is applied.

    Returns:
    pd.DataFrame: DataFrame containing filtered person data.
    str: Message indicating the action taken.
    """
    if ethnicity is None:
        message = "No specific ethnicity selected. Performing a classic run."
        return person_df, message

    if ethnicity.lower() == 'hispanic':
        filtered_df = person_df[person_df['ethnicity'] == 'Hispanic or Latino']
        message = "Filtering by Hispanic ethnicity."
    elif ethnicity.lower() == 'not_hispanic':
        person_df = person_df[person_df['ethnicity'] == 'Not Hispanic or Latino']
        message = "Filtering by Non-Hispanic ethnicity."
    else:
        message = f"Invalid ethnicity option: {ethnicity}. Performing a classic run."
        return person_df, message

    return person_df, message
def perform_hail_analysis(rs_name, rs_position, sex, person_df, mt_acaf):
    """
    Performs Hail analysis for the given SNP and sex variable.

    Args:
    rs_name (str): The name of the SNP.
    rs_position (str): The position of the SNP.
    sex (str): The sex variable to filter the data ('Male', 'Female', or None).
    person_df (pd.DataFrame): DataFrame containing person data.
    mt_acaf (str): Path to the MatrixTable file.

    Returns:
    pd.DataFrame: DataFrame containing genotype information.
    str: Message indicating the action taken.
    """
    # Initialize message
    message = ""

    # Set up Hail environment
    mt = hl.read_matrix_table(mt_acaf)

    # Filter MatrixTable for the given SNP position
    test_intervals = [rs_position]
    mt = hl.filter_intervals(mt, [hl.parse_locus_interval(x) for x in test_intervals])

    # Get genotype quality (GQ) histogram
    p = hl.plot.histogram(mt.GQ, range=(0, 100), bins=10, title='GQ Histogram', legend='GQ')

    # Add alleles message
    alleles_row = mt.aggregate_rows(hl.agg.collect(mt.alleles))
    alleles_row = alleles_row[0]
    message += f"Alleles for SNP {rs_name}: {alleles_row[0]}\n"

    # Create DataFrame with genotype information
    s = mt.aggregate_entries(hl.agg.collect(mt.s))
    AD = mt.aggregate_entries(hl.agg.collect(mt.AD))
    GT = mt.aggregate_entries(hl.agg.collect(mt.GT))
    FT = mt.aggregate_entries(hl.agg.collect(mt.FT))
    GQ = mt.aggregate_entries(hl.agg.collect(mt.GQ))
    RGQ = mt.aggregate_entries(hl.agg.collect(mt.RGQ))
    df = pd.DataFrame({'s': s, 'GT': GT, 'AD': AD, 'FT': FT, 'GQ': GQ, 'RGQ': RGQ})

    # Rename columns
    df = df.rename(columns={'s': 'id'})

    # Filter person DataFrame based on sex
    if sex.lower() == 'male':
        person_df_filtered = person_df[person_df['sex_at_birth'] == 'Male']
        message += f"Filtering by Male sex.\n"
    elif sex.lower() == 'female':
        person_df_filtered = person_df[person_df['sex_at_birth'] == 'Female']
        message += f"Filtering by Female sex.\n"
    else:
        person_df_filtered = person_df
        message += "No specific sex selected. Performing analysis on all individuals.\n"

    # Filter DataFrame based on person DataFrame
    person_df_filtered = person_df_filtered.rename(columns={'person_id': 'id'})
    person_df_filtered['id'] = person_df_filtered['id'].astype(str)
    samples = person_df_filtered['id'].tolist()
    df['id'] = df['id'].astype(str)
    df = df[df['id'].isin(samples)]
    df = pd.merge(df, person_df_filtered, on='id')

    # Save pie chart
    df['GT'].value_counts().plot(kind='pie', title='{}_{}'.format(rs_name, sex),
                                  legend=True).get_figure().savefig('./download/{}_{}_pie_GT_all_1.png'.format(rs_name, sex),
                                                                     dpi=300, facecolor='w',bbox_inches='tight')
    plt.show()

    return df, message,alleles_row
def filter_data_and_print_value_counts(df, rs_name, person_information, GQ_threshold, phecode_min):
    """
    Filter the genotype data based on genotype quality threshold and print the value counts of genotypes.

    Args:
    df (pd.DataFrame): DataFrame containing genotype data.
    rs_name (str): Name of the SNP.
    person_information (pd.DataFrame): DataFrame containing person information for conditions.
    GQ_threshold (int): Genotype quality threshold (default is 20).
    phecode_min (int): Minimum number of PheCodes in the dataset (default is 0).

    Returns:
    pd.DataFrame: Filtered DataFrame for genotype data.
    str: Message indicating the number of entries removed based on GQ threshold and PheCode minimum.
    pd.DataFrame or None: DataFrame containing conditions data if phecode_min is greater than zero, otherwise None.
    """
    # Choose samples with genotype quality >= GQ_threshold
    num_samples_removed_gq = df.shape[0] - df[df['GQ'] >= GQ_threshold].shape[0]
    df = df[df['GQ'] >= GQ_threshold]

    # Rename the 'GT' column to 'rs_name'
    df = df.rename(columns={'GT': rs_name})

    # Print value counts of genotypes
    genotype_value_counts = df[rs_name].value_counts()
    print(genotype_value_counts)

    # Sort DataFrame by 'id'
    df = df.sort_values(by=['id'])

    # Create conditions DataFrame
    conditions = pd.DataFrame(data=person_information, columns=['person_id','source_vocabulary','source_concept_code','source_concept_name','count'])
    conditions['count'] = conditions.groupby(['person_id','source_vocabulary', 'source_concept_code','source_concept_name'])['source_concept_code'].transform('count')
    conditions.sort_values(by=['person_id'], inplace=True)
    conditions.reset_index(drop=True, inplace=True)

    # Filter DataFrame based on minimum PheCode count if phecode_min is greater than zero
    if phecode_min > 0:
        list_con = conditions['source_concept_code'].value_counts().rename_axis('source_concept_code').to_frame('counts')
        list_con = list_con.reset_index()
        list_con = list_con[list_con['counts'] >= phecode_min]
        num_entries_removed_phecode = conditions.shape[0] - conditions[conditions['source_concept_code'].isin(list_con['source_concept_code'])].shape[0]
        conditions = conditions[conditions['source_concept_code'].isin(list_con['source_concept_code'])]
        conditions = conditions.reset_index(drop=True)
    else:
        num_entries_removed_phecode = 0

    # Create message
    message = f"Number of entries removed based on GQ threshold ({GQ_threshold}): {num_samples_removed_gq}\n"
    message += f"Number of entries removed based on PheCode minimum threshold ({phecode_min}): {num_entries_removed_phecode}"

    return df, message, conditions
def filter_common_ids(samples_GT, conditions, rs_name):
    """
    Keep only the IDs that have data in both samples_GT and conditions DataFrames.

    Args:
    samples_GT (pd.DataFrame): DataFrame containing genotype data with 'id' column.
    conditions (pd.DataFrame): DataFrame containing conditions data with 'id' column.
    rs_name (str): Name of the SNP.

    Returns:
    pd.DataFrame: DataFrame containing samples_GT with common IDs.
    pd.DataFrame: DataFrame containing conditions with common IDs.
    str: Message indicating the number of IDs removed and the number of unique IDs remaining in both DataFrames.
    """
    # Rename the 'person_id' column in conditions to 'id'
    conditions = conditions.rename(columns={'person_id': 'id'})

    # Convert 'id' column in samples_GT and conditions to string
    df['id'] = df['id'].astype(str)
    conditions['id'] = conditions['id'].astype(str)

    # Get unique values of 'id' columns in samples_GT and conditions
    df_unique_list = set(df['id'].unique())
    conditions_unique_list = set(conditions['id'].unique())

    # Keep only values in samples_GT_unique_list that are in conditions_unique_list
    common_ids = df_unique_list.intersection(conditions_unique_list)

    # Filter samples_GT and conditions_unique based on the common 'id' values
    df_common = df[df['id'].isin(common_ids)]
    conditions_common = conditions[conditions['id'].isin(common_ids)]

    # Select specific columns in samples_GT
    df_common = df_common[['id', rs_name]]

    # Create message
    message = f"Number of IDs removed: {len(df_unique_list) - len(common_ids)}\n"
    message += f"Number of unique IDs remaining in both DataFrames: {len(common_ids)}"

    return df_common, conditions_common, message
def filter_df_by_common_ids(df, df_common):
    """
    Keep only the IDs that are present in the df_common DataFrame from the original df.

    Args:
    df (pd.DataFrame): Original DataFrame containing genotype data.
    df_common (pd.DataFrame): DataFrame containing common IDs.

    Returns:
    pd.DataFrame: DataFrame containing df with common IDs.
    """
    # Filter the original DataFrame based on common IDs
    filtered_df = df[df['id'].isin(df_common['id'])]

    return filtered_df
def generate_genotype_frequency_table(filtered_df, rs_name):
    """
    Generate a genotype frequency table for the given DataFrame and SNP name.

    Args:
    filtered_df (pd.DataFrame): DataFrame containing genotype data.
    rs_name (str): Name of the SNP.

    Returns:
    pd.DataFrame: DataFrame containing the genotype frequency table.
    str: Message indicating the table has been created and saved.
    """
    # Count the occurrences of each genotype value for all samples
    all_genotype_counts = filtered_df[rs_name].value_counts()

    # Count the occurrences of each genotype value for male samples
    male_genotype_counts = filtered_df[filtered_df['sex_at_birth'] == 'Male'][rs_name].value_counts()

    # Count the occurrences of each genotype value for female samples
    female_genotype_counts = filtered_df[filtered_df['sex_at_birth'] == 'Female'][rs_name].value_counts()

    # Define the genotypes
    genotypes = [0, 1, 2]

    # Create Series for all_gt, male_gt, and female_gt
    all_gt = pd.Series(all_genotype_counts, name='all_gt', index=genotypes)
    male_gt = pd.Series(male_genotype_counts, name='male_gt', index=genotypes)
    female_gt = pd.Series(female_genotype_counts, name='female_gt', index=genotypes)

    # Calculate the total genotype count for all samples
    total_all_gt = all_gt.sum()

    # Calculate the total genotype count for males
    total_male_gt = male_gt.sum()

    # Calculate the total genotype count for females
    total_female_gt = female_gt.sum()

    # Check if counts for male, female, or all genotypes are less than 20
    if total_all_gt < 20 or total_male_gt < 20 or total_female_gt < 20:
        print("Counts for male, female, or all genotypes are less than 20. Changing counts to <20.")
        all_gt = all_gt.apply(lambda x: x if x >= 20 else '<20')
        male_gt = male_gt.apply(lambda x: x if x >= 20 else '<20')
        female_gt = female_gt.apply(lambda x: x if x >= 20 else '<20')

    # Replace NaN counts with 0
    all_gt.fillna(0, inplace=True)
    male_gt.fillna(0, inplace=True)
    female_gt.fillna(0, inplace=True)
    # Convert counts to integers
    all_gt = all_gt.astype(int)
    male_gt = male_gt.astype(int)
    female_gt = female_gt.astype(int)

    # Create a DataFrame to store the frequencies of each genotype, with NaN for missing values
    frequency_df = pd.DataFrame({'all_gt': all_gt, 'male_gt': male_gt, 'female_gt': female_gt})

    # Replace counts less than 20 with "<20"
    frequency_df = frequency_df.applymap(lambda x: x if isinstance(x, str) else '<20' if x < 20 else x)

    # Display the frequency DataFrame using tabulate
    table_str = tabulate(frequency_df, headers='keys', tablefmt='pretty')

    # Save the DataFrame to a CSV file
    file_path = './download/{}_frequency_of_genotype.csv'.format(rs_name)
    frequency_df.to_csv(file_path, index=False)

    # Message indicating the table has been created and saved
    message = f"Genotype frequency table has been created and saved.\n"
    message += table_str

    return frequency_df, message
def plot_genotype_proportion(filtered_df, rs_name, pop_variable):
    """
    Plot the proportion of male and female genotypes from the filtered DataFrame and save it as an image.

    Args:
    filtered_df (pd.DataFrame): DataFrame containing the filtered genotype data.
    rs_name (str): Name of the SNP.
    pop_variable (str): Name of the population.

    Returns:
    None
    """
    # Count the occurrences of each genotype value for all samples
    all_gt = filtered_df[rs_name].value_counts().tolist()

    # Count the occurrences of each genotype value for male samples
    male_gt = filtered_df[filtered_df['sex_at_birth'] == 'Male'][rs_name].value_counts().tolist()
    if len(male_gt) != 3:
        male_gt.append(0)

    # Count the occurrences of each genotype value for female samples
    female_gt = filtered_df[filtered_df['sex_at_birth'] == 'Female'][rs_name].value_counts().tolist()
    if len(female_gt) != 3:
        female_gt.append(0)

    # Total number of samples
    total_samples = sum(all_gt)

    # Calculate proportions
    all_gt_proportion = [count / total_samples * 100 for count in all_gt]
    male_gt_proportion = [count / total_samples * 100 for count in male_gt]
    female_gt_proportion = [count / total_samples * 100 for count in female_gt]

    # Define the genotypes
    genotypes = ['0', '1', '2']

    # Create DataFrame to store the proportions of each genotype
    proportion_df = pd.DataFrame({'All': all_gt_proportion, 'Male': male_gt_proportion, 'Female': female_gt_proportion}, index=genotypes)

    # Melt the DataFrame to long format for easier plotting
    melted_df = pd.melt(proportion_df.reset_index(), id_vars='index', var_name='Group', value_name='Proportion')

    # Plotting grouped bar graph
    plt.figure(figsize=(10, 6))
    sns.barplot(x='index', y='Proportion', hue='Group', data=melted_df, palette='viridis')

    # Set y-axis limits
    plt.ylim(0, 100)

    # Adding labels and title
    plt.xlabel('Genotypes')
    plt.ylabel('Proportion (%)')
    plt.title('Proportion of Male and Female Genotypes')

    # Save the figure
    plt.savefig('./download/{}_{}_Genotype_Proportions_for_Different_Sex_Groups.png'.format(rs_name, pop_variable), bbox_inches='tight', pad_inches=0.5, dpi=300, facecolor='w')

    # Show plot
    plt.show()
def transform_sex_labels_and_calculate_age(df):
    """
    Transform sex labels from 'Male' to 'M' and 'Female' to 'F' in the DataFrame,
    and calculate the age based on the date of birth.

    Args:
    df (pd.DataFrame): DataFrame containing the sex labels and date of birth.

    Returns:
    pd.DataFrame: DataFrame with transformed sex labels and calculated age.
    str: Message indicating the transformation and age calculation.
    """
    # Make a copy of the DataFrame to avoid modifying the original DataFrame
    df_copy = df.copy()

    # Map 'Male' to 'M' and 'Female' to 'F' in a new 'sex' column
    df_copy['sex'] = df_copy['sex_at_birth'].map({'Male': 'M', 'Female': 'F'})

    # Calculate age based on date of birth
    df_copy['date_of_birth'] = df_copy['date_of_birth'].astype(str).str[:4]
    df_copy['age'] = 2023 - df_copy['date_of_birth'].astype(int)

    # Add a message
    message = 'Transformed sex labels to "M" and "F" and calculated age.'

    return df_copy, message
def plot_age_distribution_for_genotypes(filtered_df, rs_name):
    """
    Generate plots for age distribution for different genotypes in males, females, and overall.

    Args:
    filtered_df (pd.DataFrame): DataFrame containing the filtered genotype data.
    rs_name (str): Name of the SNP.

    Returns:
    str: Message indicating that plots have been saved.
    """
    print(filtered_df.head())  # Debug print to check the filtered DataFrame
    genotypes = [0, 1, 2]
    sex_labels = {'M': 'Male', 'F': 'Female'}

    for genotype in genotypes:
        # Debug print to check the number of samples for each genotype
        print(f"Number of samples for genotype {genotype}: {len(filtered_df[filtered_df[rs_name] == genotype])}")

        # Skip plotting if there are no samples for the current genotype
        if len(filtered_df[filtered_df[rs_name] == genotype]) == 0:
            print(f"No samples found for genotype {genotype}. Skipping plotting.")
            continue

        # Visualize age distribution for different genotypes in males
        plt.figure(figsize=(8, 6))
        for sex in ['M', 'F']:
            sns.histplot(filtered_df[(filtered_df['sex'] == sex) & (filtered_df[rs_name] == genotype)]['age'], kde=True, label=f'{sex_labels[sex]}')
        plt.title(f'Age Distribution for Genotype {genotype} (Male vs Female)')
        plt.xlabel('Age')
        plt.ylabel('Density')
        plt.legend()
        plt.savefig(f'./download/{rs_name}_age_distribution_genotype_{genotype}_male_vs_female.png', dpi=300)
        plt.show()

        # Visualize overall age distribution for different genotypes
        plt.figure(figsize=(8, 6))
        sns.histplot(filtered_df[filtered_df[rs_name] == genotype]['age'], kde=True)
        plt.title(f'Overall Age Distribution for Genotype {genotype}')
        plt.xlabel('Age')
        plt.ylabel('Density')
        plt.savefig(f'./download/{rs_name}_age_distribution_genotype_{genotype}_overall.png', dpi=300)
        plt.show()

    message = 'Plots for age distribution for different genotypes have been saved.'
    return message
def plot_age_distribution_for_genotypes(filtered_df, rs_name, pop_variable):
    """
    Generate plots for age distribution for different genotypes in males, females, and overall.

    Args:
    filtered_df (pd.DataFrame): DataFrame containing the filtered genotype data.
    rs_name (str): Name of the SNP.
    pop_variable (str): Population variable.

    Returns:
    str: Message indicating that plots have been saved.
    """
    # Convert genotypes to string
    genotypes = [0, 1, 2]

    # Sex labels mapping
    sex_labels = {'M': 'Male', 'F': 'Female'}

    # Visualize age distribution for different genotypes in males
    plt.figure(figsize=(8, 6))
    for genotype in genotypes:
        sns.histplot(filtered_df[(filtered_df['sex'] == 'M') & (filtered_df[rs_name] == genotype)]['age'], kde=True, label=f'GT={genotype}')
    plt.title('Age Distribution in Males for Different Genotypes without sex aggregation')
    plt.xlabel('Age')
    plt.ylabel('Density')
    plt.legend()
    plt.savefig(f'./download/{rs_name}_{pop_variable}_age_Male_distribution_with_dif_Genotypes_without_sex_aggregation.png', dpi=300, facecolor='w')
    plt.show()

    # Visualize age distribution for different genotypes in females
    plt.figure(figsize=(8, 6))
    for genotype in genotypes:
        sns.histplot(filtered_df[(filtered_df['sex'] == 'F') & (filtered_df[rs_name] == genotype)]['age'], kde=True, label=f'GT={genotype}')
    plt.title('Age Distribution in Females for Different Genotypes without sex aggregation')
    plt.xlabel('Age')
    plt.ylabel('Density')
    plt.legend()
    plt.savefig(f'./download/{rs_name}_{pop_variable}_age_Female_distribution_with_dif_Genotypes_without_sex_agg.png', dpi=300, facecolor='w')
    plt.show()

    # Visualize overall age distribution for different genotypes
    plt.figure(figsize=(8, 6))
    for genotype in genotypes:
        sns.histplot(filtered_df[filtered_df[rs_name] == genotype]['age'], kde=True, label=f'GT={genotype}')
    plt.title('Overall Age Distribution for Different Genotypes without sex agg')
    plt.xlabel('Age')
    plt.ylabel('Density')
    plt.legend()
    plt.savefig(f'./download/{rs_name}_{pop_variable}_age_all_population_distribution_with_dif_Genotypes_without_sex_agg.png', dpi=300, facecolor='w')
    plt.show()

    message = f'Plots for age distribution for different genotypes have been saved.'
    return message
def merge_population_data(filtered_df, population):
    """
    Merge population data with genotype data.

    Args:
    filtered_df (pd.DataFrame): DataFrame containing the filtered genotype data.
    population (pd.DataFrame): DataFrame containing population data.

    Returns:
    pd.DataFrame: Combined DataFrame with genotype and population data.
    str: Message indicating that population data has been merged.
    str: Message indicating if any samples were removed after the merge.
    """

    # Rename the column 'person_id' to 'id' in population_new
    population_new = population.rename(columns={'person_id': 'id'})

    # Ensure 'id' column in population_new is of the same data type as in filtered_df
    population_new['id'] = population_new['id'].astype(str)

    # Merge population_new with filtered_df on 'id' column
    merged_df = pd.merge(filtered_df, population_new, on='id', how='left')

    # Drop columns ending with '_x' or '_y' (resulting from merge)
    merged_df = merged_df.loc[:, ~merged_df.columns.str.endswith('_x') & ~merged_df.columns.str.endswith('_y')]

    # Check if any samples were removed after the merge
    removed_samples = len(filtered_df) - len(merged_df)
    if removed_samples > 0:
        removed_message = f"{removed_samples} samples were removed after the merge."
    else:
        removed_message = "No samples were removed after the merge."

    # Message indicating that population data has been merged
    merge_message = "Population data has been merged with genotype data."

    return merged_df, merge_message, removed_message
def filter_and_create_dataframes(merged_df, conditions_common, icd9cm=True, icd10cm=True):
    """
    Filter the merged DataFrame based on common IDs and create separate DataFrames for ICD9CM and ICD10CM, or combine both.

    Args:
    merged_df (pd.DataFrame): Merged DataFrame containing genotype and population data.
    conditions_common (pd.DataFrame): DataFrame containing condition data.
    icd9cm (bool): Whether to create a DataFrame for ICD9CM (default is True).
    icd10cm (bool): Whether to create a DataFrame for ICD10CM (default is True).

    Returns:
    pd.DataFrame: Combined DataFrame with ICD9CM and ICD10CM data, or separate DataFrames based on user choice.
    str: Message indicating how many data points were removed and how many remain.
    """

    # Find filtered ids
    final_data_ids = merged_df['id'].tolist()

    # Filter the ids from conditions_common that are present in final_data
    filtered_ids = conditions_common[conditions_common['id'].isin(final_data_ids)]
    removed_count = len(conditions_common) - len(filtered_ids)

    if icd9cm and icd10cm:
        # Separate DataFrames for ICD9CM and ICD10CM
        conditions_unique_ICD10CM = filtered_ids[filtered_ids['source_vocabulary'] == 'ICD10CM']
        conditions_unique_ICD9CM = filtered_ids[filtered_ids['source_vocabulary'] == 'ICD9CM']

        # Save separate DataFrames
        save_conditions_unique_ICD9CM = conditions_unique_ICD9CM[['id', 'source_vocabulary', 'source_concept_code', 'count']]
        save_conditions_unique_ICD10CM = conditions_unique_ICD10CM[['id', 'source_vocabulary', 'source_concept_code', 'count']]

        # Combine ICD9CM and ICD10CM DataFrames
        conditions_unique_10_9 = pd.concat([save_conditions_unique_ICD10CM, save_conditions_unique_ICD9CM], axis=0)
        remaining_count = len(conditions_unique_10_9)

        message = f"{removed_count} Disease were removed. Disease points: {remaining_count}"

        return conditions_unique_10_9, message
    elif icd9cm:
        # Filter only ICD9CM data
        conditions_unique_ICD9CM = filtered_ids[filtered_ids['source_vocabulary'] == 'ICD9CM']
        remaining_count = len(conditions_unique_ICD9CM)

        message = f"{removed_count} Disease were removed. Disease points for ICD9CM: {remaining_count}"

        return conditions_unique_ICD9CM[['id', 'source_vocabulary', 'source_concept_code', 'count']], message
    elif icd10cm:
        # Filter only ICD10CM data
        conditions_unique_ICD10CM = filtered_ids[filtered_ids['source_vocabulary'] == 'ICD10CM']
        remaining_count = len(conditions_unique_ICD10CM)

        message = f"{removed_count} Disease were removed. Remaining Disease for ICD10CM: {remaining_count}"

        return conditions_unique_ICD10CM[['id', 'source_vocabulary', 'source_concept_code', 'count']], message
    else:
        # Combine ICD9CM and ICD10CM DataFrames if both icd9cm and icd10cm are False
        conditions_unique_ICD10CM = filtered_ids[filtered_ids['source_vocabulary'] == 'ICD10CM']
        conditions_unique_ICD9CM = filtered_ids[filtered_ids['source_vocabulary'] == 'ICD9CM']

        # Save separate DataFrames
        save_conditions_unique_ICD9CM = conditions_unique_ICD9CM[['id', 'source_vocabulary', 'source_concept_code', 'count']]
        save_conditions_unique_ICD10CM = conditions_unique_ICD10CM[['id', 'source_vocabulary', 'source_concept_code', 'count']]

        # Combine ICD9CM and ICD10CM DataFrames
        conditions_unique_10_9 = pd.concat([save_conditions_unique_ICD10CM, save_conditions_unique_ICD9CM], axis=0)
        remaining_count = len(conditions_unique_10_9)

        message = f"{removed_count} Disease were removed. Disease patients for ICD9CM and ICD10CM: {remaining_count}"

        return conditions_unique_10_9, message
def create_pie_chart_after_filtering(genotype_counts, rs_name, pop_variable):
    """
    Create a pie chart based on genotype counts.

    Args:
    genotype_counts (pd.Series): Series containing genotype counts.
    rs_name (str): Name of the SNP.
    pop_variable (str): Name of the population variable.

    Returns:
    None
    """
    # Replace values less than 20 with a placeholder
    genotype_counts = genotype_counts.apply(lambda x: x if x >= 20 else '<20')

    # Plot the pie chart
    pie_chart = genotype_counts.plot(kind='pie', title='{}'.format(rs_name), legend=True)

    # Add the genotype counts to the title
    title_with_counts = '{} (0: {}, 1: {}, 2: {})'.format(rs_name, 
                                                         genotype_counts.get(0, 0),
                                                         genotype_counts.get(1, 0),
                                                         genotype_counts.get(2, 0))
    pie_chart.set_title(title_with_counts)

    # Save the figure
    plt.savefig('./download/{}_{}_pie_GT_my_data_after_filtering.png'.format(rs_name, pop_variable), dpi=300, facecolor='w')

    # Show the plot
    plt.show()
def create_genotype_files_and_plots(merged_df, rs_name, pop_variable):
    """
    Create two files and two plots for genotype information from the merged DataFrame.
    
    Args:
    merged_df (pd.DataFrame): DataFrame containing genotype and population data.
    rs_name (str): Name of the genotype column.
    pop_variable (str): Name of the population variable.

    Returns:
    None
    """

    # Check if the merged_df is empty
    if merged_df.empty:
        print("Error: The merged DataFrame is empty.")
        return
    
    # Define the genotype values to count
    genotypes_to_count = [0, 1, 2]

    # Group by 'ancestry_pred' and aggregate the data
    aggregated_data = merged_df.groupby('ancestry_pred').agg({
        'sex': lambda x: x.value_counts().to_dict(),  # Count of each sex
    })

    # Count each genotype for each ancestry_pred group
    for genotype in genotypes_to_count:
        aggregated_data[f'M_gt_{genotype}'] = merged_df.loc[merged_df['sex'] == 'M'].groupby('ancestry_pred')[rs_name].apply(lambda x: (x == genotype).sum())
        aggregated_data[f'F_gt_{genotype}'] = merged_df.loc[merged_df['sex'] == 'F'].groupby('ancestry_pred')[rs_name].apply(lambda x: (x == genotype).sum())
    # Count each genotype for each ancestry_pred group
    for genotype in genotypes_to_count:
        aggregated_data[f'genotype_{genotype}_count'] = merged_df.groupby('ancestry_pred')[rs_name].apply(lambda x: (x == genotype).sum())

    # Ensure counts for males ('M') and females ('F') are integers and replace NaN with 0
    aggregated_data['male_count'] = aggregated_data['sex'].apply(lambda x: int(x.get('M', 0)) if isinstance(x, dict) else 0)
    aggregated_data['female_count'] = aggregated_data['sex'].apply(lambda x: int(x.get('F', 0)) if isinstance(x, dict) else 0)

    # Drop the original 'sex' column
    aggregated_data = aggregated_data.drop('sex', axis=1)

    aggregated_data.reset_index(inplace=True)
    df1 = aggregated_data
    # Now 'ancestry_pred' is a regular column
    print(aggregated_data)
    ancestry_pred = df1['ancestry_pred']
    genotypes = ['0', '1', '2']

    # Plotting
    fig, ax = plt.subplots(figsize=(10, 6))

    # Define bar width
    bar_width = 0.15

    # Define positions for each group of bars
    r1 = range(len(ancestry_pred))
    r2 = [x + bar_width for x in r1]
    r3 = [x + bar_width for x in r2]
    r4 = [x + bar_width for x in r3]
    r5 = [x + bar_width for x in r4]
    r6 = [x + bar_width for x in r5]

    # Plot bars for each genotype and gender combination
    plt.bar(r1, df1['M_gt_0'], color='b', width=bar_width, edgecolor='grey', label='M_gt_0')
    plt.bar(r2, df1['F_gt_0'], color='g', width=bar_width, edgecolor='grey', label='F_gt_0')
    plt.bar(r3, df1['M_gt_1'], color='r', width=bar_width, edgecolor='grey', label='M_gt_1')
    plt.bar(r4, df1['F_gt_1'], color='c', width=bar_width, edgecolor='grey', label='F_gt_1')
    plt.bar(r5, df1['M_gt_2'], color='m', width=bar_width, edgecolor='grey', label='M_gt_2')
    plt.bar(r6, df1['F_gt_2'], color='y', width=bar_width, edgecolor='grey', label='F_gt_2')

    # Add xticks on the middle of the group bars
    plt.xlabel('Ancestry prediction', fontweight='bold')
    plt.xticks([r + bar_width * 2.5 for r in range(len(ancestry_pred))], ancestry_pred)
    plt.title(f'Number of genotype in all ancestry for {rs_name}')
    # Add y label
    plt.ylabel('Count')

    # Create legend & Show graphic
    plt.legend()
    # save it 
    plt.savefig('./download/{}_{}_Number of genotype in all ancestry.pdf'.format(rs_name,pop_variable),dpi=300,facecolor='w')
    plt.show()

    # Define the genotype values to count
    genotypes_to_count = [0, 1, 2]

    # Group by 'ancestry_pred' and aggregate the data
    aggregated_data_other = merged_df.groupby('ancestry_pred_other').agg({
        'sex': lambda x: x.value_counts().to_dict(),  # Count of each sex
    })

    # Count each genotype for each ancestry_pred group
    for genotype in genotypes_to_count:
        aggregated_data_other[f'M_gt_{genotype}'] = merged_df.loc[merged_df['sex'] == 'M'].groupby('ancestry_pred_other')[rs_name].apply(lambda x: (x == genotype).sum())
        aggregated_data_other[f'F_gt_{genotype}'] = merged_df.loc[merged_df['sex'] == 'F'].groupby('ancestry_pred_other')[rs_name].apply(lambda x: (x == genotype).sum())
    # Count each genotype for each ancestry_pred group
    for genotype in genotypes_to_count:
        aggregated_data_other[f'genotype_{genotype}_count'] = merged_df.groupby('ancestry_pred_other')[rs_name].apply(lambda x: (x == genotype).sum())

    # Ensure counts for males ('M') and females ('F') are integers and replace NaN with 0
    aggregated_data_other['male_count'] = aggregated_data_other['sex'].apply(lambda x: int(x.get('M', 0)) if isinstance(x, dict) else 0)
    aggregated_data_other['female_count'] = aggregated_data_other['sex'].apply(lambda x: int(x.get('F', 0)) if isinstance(x, dict) else 0)

    # Drop the original 'sex' column
    aggregated_data_other = aggregated_data_other.drop('sex', axis=1)

    aggregated_data_other.reset_index(inplace=True)
    df1 = aggregated_data_other
    # Now 'ancestry_pred' is a regular column
    print(aggregated_data_other)
    ancestry_pred = df1['ancestry_pred_other']
    genotypes = [0, 1, 2]

    # Plotting
    fig, ax = plt.subplots(figsize=(10, 6))

    # Define bar width
    bar_width = 0.15

    # Define positions for each group of bars
    r1 = range(len(ancestry_pred))
    r2 = [x + bar_width for x in r1]
    r3 = [x + bar_width for x in r2]
    r4 = [x + bar_width for x in r3]
    r5 = [x + bar_width for x in r4]
    r6 = [x + bar_width for x in r5]

    # Plot bars for each genotype and gender combination
    plt.bar(r1, df1['M_gt_0'], color='b', width=bar_width, edgecolor='grey', label='M_gt_0')
    plt.bar(r2, df1['F_gt_0'], color='g', width=bar_width, edgecolor='grey', label='F_gt_0')
    plt.bar(r3, df1['M_gt_1'], color='r', width=bar_width, edgecolor='grey', label='M_gt_1')
    plt.bar(r4, df1['F_gt_1'], color='c', width=bar_width, edgecolor='grey', label='F_gt_1')
    plt.bar(r5, df1['M_gt_2'], color='m', width=bar_width, edgecolor='grey', label='M_gt_2')
    plt.bar(r6, df1['F_gt_2'], color='y', width=bar_width, edgecolor='grey', label='F_gt_2')

    # Add xticks on the middle of the group bars
    plt.xlabel('Ancestry prediction', fontweight='bold')

    plt.xticks([r + bar_width * 2.5 for r in range(len(ancestry_pred))], ancestry_pred)

    plt.title(f'Number of genotype in all ancestry and other for {rs_name}')
    # Add y label
    plt.ylabel('Count')

    # Create legend & Show graphic
    plt.legend()
    # save it 
    plt.savefig('./download/{}_{}_Number of genotype in all ancestry with other.pdf'.format(rs_name,pop_variable),dpi=300,facecolor='w')
    plt.show()
    # Define a function to replace values less than 20 with '<20'
    def replace_less_than_20(value):
        return '<20' if value < 20 else value

    # Iterate over the columns of the first dataframe (aggregated_data)
    for column in aggregated_data.columns[1:]:  # Exclude the 'ancestry_pred' column
        aggregated_data[column] = aggregated_data[column].apply(replace_less_than_20)

    # Iterate over the columns of the second dataframe (aggregated_data_other)
    for column in aggregated_data_other.columns[1:]:  # Exclude the 'ancestry_pred' column
        aggregated_data_other[column] = aggregated_data_other[column].apply(replace_less_than_20)

    # Save to CSV
    aggregated_data.to_csv('download/{}_{} Count of Genotypes in ancestry.csv'.format(rs_name, pop_variable), index=False)
    aggregated_data_other.to_csv('download/{}_{} Count of Genotypes in ancestry with other.csv'.format(rs_name, pop_variable), index=False)
def generate_genotype_plots(merged_df, rs_name, pop_variable):
    # Set the color palette for each genotype and specify the order of legend labels
    
    genotype_colors = {0: 'purple', 1: 'orange', 2: 'blue'}
    genotype_order = [0, 1, 2]
    
    # Combine all genotypes for the violin plot
    combined_genotypes = merged_df[merged_df[rs_name].isin(genotype_colors.keys())]

    # Melt the DataFrame for easier plotting
    melted_data = pd.melt(combined_genotypes, id_vars=rs_name, value_vars=[f'principal_component_{i}' for i in range(1, 17)])

    # Initialize the figure and axes for the combined violin plot
    fig, ax = plt.subplots(figsize=(12, 8))

    # Create a violin plot for all genotypes
    sns.violinplot(x="variable", y="value", hue=rs_name, data=melted_data, palette=genotype_colors.values(), inner='quartile', ax=ax)

    # Set chart title and labels
    ax.set_title(f'Violin Plot - PCA Values for All Genotypes for {rs_name}')
    ax.set_xlabel('Principal Components')
    ax.set_ylabel('PCA Values')
    ax.set_xticks(range(0, 16))
    ax.set_xticklabels(['PCA{}'.format(i) for i in range(1, 17)])
    ax.tick_params(axis='x', rotation=90)

    # Create a custom legend with the correct color-number combination
    legend_elements = [plt.Line2D([0], [0], marker='o', color='w', label=f'Genotype {key}', markerfacecolor=value, markersize=10) for key, value in genotype_colors.items()]

    # Add the custom legend
    ax.legend(handles=legend_elements, title=rs_name, loc='upper right', bbox_to_anchor=(1.12, 1), labels=genotype_order)

    # Save the combined plot
    plt.savefig(f'./download/{rs_name}_All_Genotypes_Violin_Plot {pop_variable}.png', dpi=300, facecolor='w')
def create_genotype_boxplot(merged_df, rs_name, pop_variable):
    """
    Create a boxplot and swarmplot for genotypes.

    Args:
    merged_df (pd.DataFrame): DataFrame containing genotype and population data.
    rs_name (str): Name of the genotype column.
    pop_variable (str): Name of the population variable.

    Returns:
    None
    """
    
    # Convert genotype values to different colors
    genotype_colors = {0: 'blue', 1: 'red', 2: 'orange'}

    # Create a boxplot and swarmplot
    plt.figure(figsize=(12, 6))
    sns.boxplot(x='sex', y='age', data=merged_df, hue=rs_name, palette=genotype_colors)

    # Add swarmplot (optional)
    # sns.swarmplot(x='sex', y='age', data=all_data.sample(100), color='green', size=3, marker='o', label='Random Samples')

    plt.title(f'Boxplot of Age by Sex and Genotype for {rs_name}')
    plt.legend()

    # Save the plot
    plt.savefig(f'./download/{rs_name}_{pop_variable}_Boxplot_Swarmplot.png', dpi=300, facecolor='w')

    # Show the plot
    plt.show()
def read_data_information_file(file_path):
    try:
        with open(file_path, "r") as file:
            data = file.read()
        return data
    except FileNotFoundError:
        return "File not found. Please make sure the file path is correct."
def perform_umap_clustering(data, num_clusters=6, random_state=42, theme="fire"):
    print("Performing UMAP dimensionality reduction...")
    

    sampled_data = data.sample(n=len(data['id']), random_state=random_state, replace=False)

    # Extract features and target
    X, y = sampled_data.drop("id", axis=1), sampled_data["id"]
    
    # Select only PCA columns 1-8
    pca_columns = [f'principal_component_{i}' for i in range(1, 9)]
    X_pca = X[pca_columns]
    
    # Perform UMAP dimensionality reduction
    umap_model = umap.UMAP(min_dist=0.01, n_neighbors=15)  # Set min_dist to 0.01
    manifold = umap_model.fit_transform(X_pca)
    
    print("Performing KMeans clustering...")
    # Perform KMeans clustering on the reduced data
    kmeans = KMeans(n_clusters=num_clusters, random_state=random_state)
    clusters = kmeans.fit_predict(manifold)
    
    print("Plotting UMAP visualization...")
    # Create a new figure
    plt.figure(figsize=(8, 6))
    
    # Define colors consistent with the population index
    colors = ['blue', 'purple','red','orange','green','brown']
    
    # Create a new column 'color' based on ancestry_pred
    sampled_data['color'] = [colors[pop_index.index(ancestry)] for ancestry in sampled_data['ancestry_pred']]
    
    # Map cluster labels to colors
    cluster_colors = sampled_data['color'].tolist()
    
#     # Plot the UMAP visualization with cluster labels and legends
#     umap.plot.points(umap_model, labels=np.array(cluster_colors), show_legend=True, color_key_cmap='Paired', background='white')
    
#     # Save the UMAP clustering plot
#     plot_path = f"./download/{rs_name}_{pop_variable}_umap_clustering_plot_1.pdf"
#     plt.savefig(plot_path, format='pdf')
#     plt.show()
    plt.close()
    
    # Add x and y coordinates to sample_df and sort by id
    sample_df = sampled_data[['id',rs_name, 'ancestry_pred', 'color']].assign(cluster=clusters)
    sample_df['x_axis'] = manifold[:, 0]
    sample_df['y_axis'] = manifold[:, 1]
    sample_df = sample_df.sort_values(by='id').reset_index(drop=True)
    
    return clusters, sample_df
# Accumulate messages
all_messages_first = ""
bucket, genomic_location = initialize_hail_environment()
# Output the bucket and genomic location
print("Bucket:", bucket)
all_messages_first += f"**Address of your Bucket:**\n{bucket}\n\n"
print("Genomic Location:", genomic_location)
all_messages_first += f"**Your Genomic Location that you using:**\n{genomic_location}\n\n"
person_information_all, message = read_person_information()
print(message)
all_messages_first += f"**Download information about the patient's conditions:**\n{message}\n\n"
# print(person_information_all.head())
person_df_all, message = read_person_information_df()
print(message)
all_messages_first += f"**Download information about the patients:**\n{message}\n\n"
# print(person_df_all.head())
bucket = os.getenv('WORKSPACE_BUCKET')
copy_files_to_local(bucket)
zip_code_socioeconomic_df, message = read_zip_code_socioeconomic_data()
print(message)
all_messages_first+= f"**Download Zip-Code information about the patients:**\n{message}\n\n"
# print(zip_code_socioeconomic_df.head())

# Set Your Variables

In [ ]:
all_messages = ""
all_messages+=all_messages_first
rs_name = 'rs184384746' # SNP's Name
rs_position = 'chr3:57192122-57192123' # position 
sex = 'None'  # Choose the sex variable ('Male', 'Female', or None)
pop_variable = 'all'  # Choose the population variable ['afr', 'amr', 'eas','eur', 'mid', 'sas', 'all']
ethnicity = 'None'  # Choose the ethnicity ('Hispanic or Latino' or 'Not Hispanic or Latino'), or None for classic run
reference_allele=None
alternative_allele=None
GQ_threshold=20
phecode_min=0
icd9cm=True
icd10cm=True
# Create messages
informations = f"SNP's Name: {rs_name}\nSNP's Position: {rs_position}\nReference Allele: {reference_allele}\nAlternative Allele: {alternative_allele}\nChoice of Sex at Birth: {sex}\nEthnisity: {ethnicity}\nPhecode minimum: {phecode_min}\nGQ (Genome Quality) Threshold: {GQ_threshold}\nUsage of ICD9CM (International Classification of Diseases): {icd9cm}\nUsage of ICD10CM (International Classification of Diseases): {icd10cm}\n"
all_messages += f"**SNP Informations:**\n{informations}\n\n"
# Print the text variable
print(all_messages)

# Start of the Analysis

In [ ]:
person_df, person_information, race_counts, hispanic_counts, population, race_counts_message, hispanic_counts_message, population_message, max_probability_message = run_analysis()
print(race_counts_message)
print(hispanic_counts_message)
print(population_message)
print(max_probability_message)
all_messages += f"**Race Counts:**\n{race_counts_message}\n\n"
all_messages += f"**Hispanic Counts:**\n{hispanic_counts_message}\n\n"
all_messages += f"**Population Message:**\n{population_message}\n\n"
all_messages += f"**Max Probability Message:**\n{max_probability_message}\n\n"
message = plot_pca_graphs(population)
print(message)
all_messages += f"**PCA Graphs Plotting Message:**\n{message}\n\n"
pca, message = perform_pca(population)
print(message)
all_messages += f"**PCA Perform Message:**\n{message}\n\n"
population, person_df, person_information, message = prepare_genotyping_data(pop_variable, population, person_df, person_information)
print(message)
all_messages += f"**Genotyping Data Preparation Message:**\n{message}\n\n"
# print(population.head())
# print(person_df.head())
# print(person_information.head())
print(len(person_df['person_id']))
person_df, message = remove_related_samples(person_df)
print(message)
all_messages += f"**Remove relatedness from the dataset:**\n{message}\n\n"
# print(person_df.head())
print(len(person_df['person_id']))
# ethnicity = 'Hispanic'
person_df, message = filter_by_ethnicity(person_df, ethnicity)
print(message)
all_messages += f"**Filter for the ethnicity that you choose:**\n{message}\n\n"
# print(person_df.head())

mt_acaf = os.getenv("WGS_ACAF_THRESHOLD_MULTI_HAIL_PATH")
mt = hl.read_matrix_table(mt_acaf)
df, message,alleles_row = perform_hail_analysis(rs_name, rs_position, sex, person_df, mt_acaf)
print(message)
all_messages += f"**Perform Hail Analysis:**\n{message}\n\n"
# print(df.head())
df, message = transform_genotypes(df, alleles_row, reference_allele,alternative_allele)
df['GT'].value_counts()

genotype_counts = df['GT'].value_counts()
create_pie_chart(genotype_counts, rs_name, pop_variable)

print(message)
# print(df.head())
all_messages += f"**Transform the alleles:**\n{message}\n\n"
df, message, conditions = filter_data_and_print_value_counts(df, rs_name, person_information, GQ_threshold, phecode_min)
print(message)
all_messages += f"**Filter data and find the Counts:**\n{message}\n\n"
df_common, conditions_common, message = filter_common_ids(df, conditions, rs_name)
print(message)
all_messages += f"**Find the common IDs:**\n{message}\n\n"
filtered_df = filter_df_by_common_ids(df, df_common)
genotype_frequency_table, message = generate_genotype_frequency_table(filtered_df, rs_name)
print(message)
all_messages += f"**Generate the Genotype Frequency:**\n{message}\n\n"
# Check if the filtered_df[rs_name] column has at least three unique genotype values
if filtered_df[rs_name].nunique() >= 3:
    plot_genotype_proportion(filtered_df, rs_name, pop_variable)
else:
    print(f"The number of unique genotype values for {rs_name} is less than 3. Cannot plot.")
filtered_df_transformed, message = transform_sex_labels_and_calculate_age(filtered_df)
print(message)
all_messages += f"**Create Sex and Age columns:**\n{message}\n\n"
# print(filtered_df_transformed)
plot_age_distribution_for_genotypes(filtered_df_transformed, rs_name,pop_variable)
message = plot_age_distribution_for_genotypes(filtered_df_transformed, rs_name, pop_variable)
print(message)
all_messages += f"**Create graphs about the Age Distribution:**\n{message}\n\n"
merged_df, merge_message, removed_message = merge_population_data(filtered_df_transformed, population)
print(merge_message)
print(removed_message)
all_messages += f"**Merging the Dataframes:**\n{merge_message}\n\n"
all_messages += f"**Removed IDs:**\n{removed_message}\n\n"
# print(merged_df)
combined_conditions_df, message = filter_and_create_dataframes(merged_df, conditions_common, icd9cm, icd10cm)
print(message)
all_messages += f"**Create the final Dataframes:**\n{message}\n\n"
# print(combined_conditions_df)
save('data','{}_{}_Genotype_infomation.csv'.format(pop_variable,rs_name),merged_df)
combined_conditions_df = combined_conditions_df[['id','source_vocabulary', 'source_concept_code', 'count']]
# ICD-9 + ICD-10 all conditions
save('data',f'{pop_variable}_{rs_name}_Conditions.csv',combined_conditions_df)
# Count the occurrences of each genotype value
genotype_counts = merged_df[rs_name].value_counts()
create_pie_chart_after_filtering(genotype_counts, rs_name, pop_variable)
create_genotype_files_and_plots(merged_df, rs_name, pop_variable)
generate_genotype_plots(merged_df, rs_name, pop_variable)
create_genotype_boxplot(merged_df, rs_name, pop_variable)
all_messages+= f"**Print the column names of the file with the covariates information:\n{merged_df.columns}**\n\n"
all_messages+= f"**Print the column names of the file with the conditions information:\n{combined_conditions_df.columns}**\n\n"
title = f"**PheWAS analysis for the SNP:{rs_name.upper()} AND the population is {pop_variable.upper()}**"
# Create information about data section
data_information = "**Information about Data:**\n\n"
data_information += f"**Start of the ANALYSIS:** Run the main pipeline**\n"
data_information += f"**Outputs:**\n\n{all_messages}"
# Combine all sections
final_text = f"{title.center(50)}\n\n"
final_text += data_information
# Write to text file
with open(f"./download/{rs_name}_{pop_variable}_data_information.txt", "w") as file:
    file.write(final_text)
file_path = f"./download/{rs_name}_{pop_variable}_data_information.txt"
data = read_data_information_file(file_path)
print(data)

# UMAP

In [ ]:
pca = merged_df[['id',rs_name,'ancestry_pred','principal_component_1',
       'principal_component_2', 'principal_component_3',
       'principal_component_4', 'principal_component_5',
       'principal_component_6', 'principal_component_7',
       'principal_component_8', 'principal_component_9',
       'principal_component_10', 'principal_component_11',
       'principal_component_12', 'principal_component_13',
       'principal_component_14', 'principal_component_15',
       'principal_component_16']]

pop_index = ['afr', 'amr', 'eas', 'eur', 'mid', 'sas']
cluster_labels, sample_df = perform_umap_clustering(pca, num_clusters=len(pop_index))
sample_df = sample_df.reset_index(drop=True)
sample_df[rs_name].value_counts()

# Counter to keep track of the number of legends plotted
legend_count = 0

# Plot the main scatterplot with smaller dot sizes
for ancestry, group in sample_df.groupby('ancestry_pred'):
    plt.scatter(group['x_axis'], group['y_axis'], color=group['color'], label=ancestry, alpha=0.5, s=10)
    legend_count += 1

# Add legend with larger dot sizes and position it to the top-right corner with an anchor
legend = plt.legend(loc='upper right', bbox_to_anchor=(1.11, 1), prop={'size': 12})

# Adjust the size of legend markers
for handle in legend.legendHandles:
    handle.set_sizes([36])

# Define marker styles for different values of rs_name
marker_styles = {0: '', 1: '*', 2: 'v'}


# Plot points based on the value of rs_name
for _, row in sample_df.iterrows():
    if row[rs_name] == 2:
        plt.scatter(row['x_axis'], row['y_axis'], color='black', marker=marker_styles[row[rs_name]], alpha=0.5, s=40)

# Add labels and title
plt.xlabel('UMAP component 1')
plt.ylabel('UMAP component 2')
plt.title(f'UMAP for {rs_name}')

# Add legend with larger dot sizes and position it to the top-right corner with an anchor
legend = plt.legend(loc='upper right', bbox_to_anchor=(1.11, 1), prop={'size': 12})

# Adjust the size of legend markers
for handle in legend.legendHandles:
    handle.set_sizes([36])

# Make the figure slightly bigger
plt.gcf().set_size_inches(13, 8)

# Save the plot as PDF
plot_path = f"./download/{rs_name}_{pop_variable}_umap_clustering_plot_with_markers.pdf"
plt.savefig(plot_path, format='pdf')

# Show plot
plt.show()
plt.close()


# Plot the main scatterplot with smaller dot sizes
for ancestry, group in sample_df.groupby('ancestry_pred'):
    plt.scatter(group['x_axis'], group['y_axis'], color=group['color'], label=ancestry, alpha=0.5, s=10)

# Add labels and title
plt.xlabel('UMAP component 1')
plt.ylabel('UMAP component 2')
plt.title(f'UMAP for {rs_name}')

# Add legend with larger dot sizes and position it to the top-right corner with an anchor
legend = plt.legend(loc='upper right', bbox_to_anchor=(1.11, 1), prop={'size': 12})

# Adjust the size of legend markers
for handle in legend.legendHandles:
    handle.set_sizes([36])

# Make the figure slightly bigger
plt.gcf().set_size_inches(13, 8)

# Save the plot as PDF
plot_path = f"./download/{rs_name}_{pop_variable}_umap_clustering_plot_classical.pdf"
plt.savefig(plot_path, format='pdf')

# Show plot
plt.show()
plt.close()

# Filter the data for genotype 1
genotype_1_df = sample_df[sample_df[rs_name] == 1]

# Plot the scatterplot for genotype 1
plt.scatter(genotype_1_df['x_axis'], genotype_1_df['y_axis'], color='gray', marker='*', alpha=0.5, s=40)

# Add labels and title
plt.xlabel('UMAP component 1')
plt.ylabel('UMAP component 2')
plt.title(f'UMAP for {rs_name} with Genotype 1')

# Make the figure slightly bigger
plt.gcf().set_size_inches(13, 8)

# Save the plot as PDF
plot_path = f"./download/{rs_name}_{pop_variable}_umap_clustering_plot_genotype_1.pdf"
plt.savefig(plot_path, format='pdf')

# Show plot
plt.show()
plt.close()

# Filter the data for genotype 2
genotype_2_df = sample_df[sample_df[rs_name] == 2]

# Check if genotype_2_df is not empty
if not genotype_2_df.empty:
    # Plot the scatterplot for genotype 2
    plt.scatter(genotype_2_df['x_axis'], genotype_2_df['y_axis'], color='black', marker='v', alpha=0.5, s=40)

    # Add labels and title
    plt.xlabel('UMAP component 1')
    plt.ylabel('UMAP component 2')
    plt.title(f'UMAP for {rs_name} with Genotype 2')

    # Make the figure slightly bigger
    plt.gcf().set_size_inches(13, 8)

    # Save the plot as PDF
    plot_path = f"./download/{rs_name}_{pop_variable}_umap_clustering_plot_genotype_2.pdf"
    plt.savefig(plot_path, format='pdf')

    # Show plot
    plt.show()
    plt.close()
else:
    print("Genotype 2 data is empty. Skipping plot generation.")
